# Sistema Avanzado de Trading y Análisis de Opciones sobre SPY

## Proyecto Completo de Ingeniería Financiera Cuantitativa

Este notebook implementa un sistema completo de análisis y trading de opciones sobre SPY, incluyendo:

- **Conexión a Interactive Brokers** para datos en tiempo real
- **Implementación propia de Black-Scholes** con cálculo de todas las griegas
- **Análisis de volatilidad implícita** y visualización de volatility surfaces
- **Delta hedging profesional** con simulación histórica
- **Estrategias de trading** (Long Straddle) con backtesting
- **Análisis comparativo** entre cálculos propios y datos del broker
- **Simulación de órdenes** y análisis de riesgo de ejecución

 Nomenclatura Importante

Este notebook usa la siguiente nomenclatura consistente:
- `S`, `K`, `T`, `r`, `q`, `sigma` (variables de Black-Scholes)
- `ib` para la conexión a Interactive Brokers
- `df_chain` para la cadena de opciones
- Funciones standalone: `bs_price()`, `bs_greeks_manual()`, `implied_vol_bisect()`

---

**Autor**: Sistema de Trading Cuantitativo  
**Fecha**: Diciembre 2024  
**Versión**: 2.0



# Parte 1: Configuración y Setup

## Mapa de Integración

| Variable Original | Tipo | Uso en Código Nuevo |
|-------------------|------|---------------------|
| `S` | float | Precio spot del subyacente - **MANTENER** |
| `K` | float | Strike price - **MANTENER** |
| `T` | float | Tiempo hasta vencimiento (años) - **MANTENER** |
| `r` | float | Tasa libre de riesgo - **MANTENER** |
| `q` | float | Dividend yield - **MANTENER** |
| `sigma` | float | Volatilidad - **MANTENER** |
| `ib` | IB object | Conexión a Interactive Brokers - **MANTENER** |
| `df_chain` | DataFrame | Cadena de opciones - **MANTENER** |
| `expiry` | str | Fecha de vencimiento (YYYYMMDD) - **MANTENER** |

| Función | Descripción | Uso en Código Nuevo |
|---------|-------------|---------------------|
| `bs_price(S, K, T, r, q, sigma, right)` | Calcula precio BS |  Reutilizar |
| `bs_greeks_manual(S, K, T, r, q, sigma, right)` | Calcula griegas |  Reutilizar |
| `implied_vol_bisect(price_mkt, S, K, T, r, q, right, ...)` | Calcula IV |  Reutilizar |
| `get_risk_free_rate_yahoo(default=0.045)` | Obtiene tasa libre de riesgo |  Reutilizar |
| `plot_surface_v15(df, ticker)` | Visualiza volatility surface |  Reutilizar |

## Objetivos Ya Implementados

1.  **Objetivo 1.1**: Conexión a Interactive Brokers desde Python
2.  **Objetivo 1.2**: Definir contratos de opciones sobre SPY y obtener cadenas
3.  **Objetivo 1.3**: Estimación de volatilidad implícita (método bisección)
4.  **Objetivo 1.4**: Visualización de volatility smiles
5.  **Objetivo 1.5**: Cálculo de griegas (Delta, Gamma, Theta, Vega, Rho)

## Objetivos Pendientes a Implementar

1.  **Objetivo 1.6**: Evolución temporal histórica de opciones (griegas y payoff)
2.  **Objetivo 1.7**: Función de cobertura profesional (delta-hedging detallado)
3.  **Objetivo 1.8**: Simulación de envío de órdenes (slippage, latencia, bid-ask)
4.  **Objetivo 2.1**: Estrategia Long Straddle periódico con backtesting
5.  **Objetivo 2.2**: Versión delta-hedged del straddle
6.  **Objetivo 2.3**: Análisis exhaustivo de P&L histórico
7.  **Objetivo 2.4**: Simulación combo vs patas separadas (legging risk)
8.  **Objetivo 2.5**: Neutralizar Delta con otra opción
9.  **Objetivo 2.6**: Reflexión técnica SPY vs SPX

## Principios de Integración

1. **PRESERVAR**: Todo el código original se mantiene intacto
2. **REUTILIZAR**: Las funciones existentes se usan en lugar de reescribirlas
3. **EXTENDER**: Nuevas funcionalidades se añaden sin modificar las existentes
4. **CONSISTENCIA**: Nomenclatura original se mantiene en todo el código nuevo
5. **DOCUMENTAR**: Cada sección nueva explica qué reutiliza del código original



## Configuración Inicial

Este notebook es **completamente autónomo** e incluye todas las funciones necesarias.

El notebook incluye:
- Configuración e imports
- Funciones de Black-Scholes (`bs_price`, `bs_greeks_manual`)
- Cálculo de volatilidad implícita (`implied_vol_bisect`)
- Conexión a Interactive Brokers (opcional, con modo simulación)
- Todas las extensiones y objetivos completados

**Las siguientes celdas configuran el entorno y verifican que todo esté listo.**



In [2]:
# Verificación y carga opcional del notebook original
import os

notebook_path = 'EJERCICIO_MIAX_2025.ipynb'

# Verificar si el notebook original existe
if os.path.exists(notebook_path):
    print(f"[INFO] Notebook original encontrado: {notebook_path}")
    print("  Si deseas cargar sus funciones, ejecuta en una celda separada:")
    print(f"  %run {notebook_path}")
    print("\n  O ejecuta este notebook en el mismo kernel donde ejecutaste el notebook original.")
else:
    print(f"[INFO] Notebook original no encontrado en: {notebook_path}")
    print("  No es necesario: este notebook incluye todas las funciones requeridas.")

print("\n" + "="*60)
print("NOTA: Este notebook es autónomo. Las funciones se cargarán automáticamente.")
print("="*60 + "\n")


[INFO] Notebook original encontrado: EJERCICIO_MIAX_2025.ipynb
  Si deseas cargar sus funciones, ejecuta en una celda separada:
  %run EJERCICIO_MIAX_2025.ipynb

  O ejecuta este notebook en el mismo kernel donde ejecutaste el notebook original.

NOTA: Este notebook es autónomo. Las funciones se cargarán automáticamente.



In [3]:
# Imports Necesarios para las Extensiones

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('default')
        
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

# Crear directorios si no existen
import os
os.makedirs('images/greeks_evolution', exist_ok=True)
os.makedirs('images/pnl_analysis', exist_ok=True)

print("[OK] Imports y configuración completados\n")

# Intentar cargar el notebook original si está disponible
notebook_original_cargado = False
try:
    from IPython import get_ipython
    ipython = get_ipython()
    
    if ipython is not None:
        # Verificar si existe el archivo
        notebook_path = 'EJERCICIO_MIAX_2025.ipynb'
        if os.path.exists(notebook_path):
            print("[INFO] Intentando cargar funciones del notebook original...")
            try:
                # Intentar usar %run si estamos en IPython/Jupyter
                ipython.run_line_magic('run', notebook_path)
                notebook_original_cargado = True
                print(f"[OK] Notebook original cargado exitosamente: {notebook_path}\n")
            except Exception as e:
                print(f"[AVISO] No se pudo cargar automáticamente con %run: {e}")
                print("  Las funciones compatibles se cargarán automáticamente.\n")
except Exception as e:
    pass

# Verificación de dependencias del código original
print("[INFO] Verificando que las funciones del código original están disponibles...\n")

# Lista de funciones críticas que deben existir
funciones_requeridas = [
    'bs_price',
    'bs_greeks_manual', 
    'implied_vol_bisect',
    'get_risk_free_rate_yahoo'
]

# También verificar variables opcionales que pueden venir del notebook original
variables_opcionales = ['S', 'K', 'T', 'r', 'q', 'sigma', 'ib', 'df_chain', 'expiry']

funciones_disponibles = []
funciones_faltantes = []

for func in funciones_requeridas:
    try:
        if func in globals() or func in dir():
            funciones_disponibles.append(func)
            print(f"  [OK] {func}: Disponible")
        else:
            funciones_faltantes.append(func)
            print(f"  [FALTA] {func}: FALTANTE")
    except:
        funciones_faltantes.append(func)
        print(f"  [FALTA] {func}: FALTANTE")

# Verificar variables opcionales
variables_disponibles = []
for var in variables_opcionales:
    try:
        if var in globals() or var in dir():
            variables_disponibles.append(var)
    except:
        pass

if variables_disponibles:
    print(f"\n  [INFO] Variables del notebook original disponibles: {', '.join(variables_disponibles)}")

print()  # Línea en blanco

if funciones_faltantes:
    print("[ADVERTENCIA] Algunas funciones no están disponibles.")
    print(f"   Funciones faltantes: {', '.join(funciones_faltantes)}")
    print("\n   SOLUCIONES:")
    print("   1. Ejecuta primero todas las celdas del notebook EJERCICIO_MIAX_2025.ipynb")
    print("   2. O ejecuta este notebook en el mismo kernel donde ejecutaste el notebook original")
    print("   3. O las funciones compatibles se cargarán automáticamente en las siguientes celdas")
else:
    print("[OK] Todas las funciones del código original están disponibles.")
    print("   Puedes continuar con las extensiones.")
    
if notebook_original_cargado:
    print("\n   [INFO] Las funciones fueron cargadas desde el notebook original.")
    
print()



[OK] Imports y configuración completados

[INFO] Intentando cargar funciones del notebook original...
Conectado: True
Server version: 176
Managed accounts: ['DUM956039']
Resumen de cadenas disponibles:
exchange tradingClass multiplier  numExpirations  numStrikes
    AMEX         2SPY        100               3           3
    AMEX          SPY        100              33         428
    BATS         2SPY        100               3           3
    BATS          SPY        100              33         428
     BOX         2SPY        100               3           3
     BOX          SPY        100              33         428
    CBOE         2SPY        100               3           3
    CBOE          SPY        100              33         428
   CBOE2         2SPY        100               3           3
   CBOE2          SPY        100              33         428
    EDGX         2SPY        100               3           3
    EDGX          SPY        100              33         428
 EMER

Error 200, reqId 5: No se encuentra definici\u00f3n del activo solicitado, contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20260311', strike=10.01, right='C', multiplier='100', exchange='SMART')
Error 200, reqId 6: No se encuentra definici\u00f3n del activo solicitado, contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20260311', strike=10.01, right='P', multiplier='100', exchange='SMART')
Error 200, reqId 9: No se encuentra definici\u00f3n del activo solicitado, contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20260311', strike=10010.0, right='C', multiplier='100', exchange='SMART')
Error 200, reqId 10: No se encuentra definici\u00f3n del activo solicitado, contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20260311', strike=10010.0, right='P', multiplier='100', exchange='SMART')
Unknown contract: Option(symbol='SPY', lastTradeDateOrContractMonth='20260311', strike=10.01, right='C', multiplier='100', exchange='SMART')
Unknown contract: Opti


Ejemplo de contratos cualificados (conId/localSymbol ya resueltos):
                     | conId=0
                     | conId=0
  2SPY  260311C00616000 | conId=841263439
  2SPY  260311P00616000 | conId=841263434
                     | conId=0
                     | conId=0
[OK] Estructura de carpetas de outputs creada/verificada
Cadena elegida: exchange=SMART, tradingClass=SPY, multiplier=100
Expiry seleccionado: 20260112

Contratos reales recibidos para expiry=20260112: 304
  expiry right  strike     conId           localSymbol exchange tradingClass
20260112     C   680.0 842128693 SPY   260112C00680000    SMART          SPY
20260112     C   750.0 842129117 SPY   260112C00750000    SMART          SPY
20260112     C   765.0 842129202 SPY   260112C00765000    SMART          SPY
20260112     P   630.0 842129283 SPY   260112P00630000    SMART          SPY
20260112     P   675.0 842129525 SPY   260112P00675000    SMART          SPY
20260112     C   565.0 842745140 SPY   260112C00565000 

Error 10091, reqId 8: Parte de los datos de mercado solicitados requiere suscripciones adicionales para API. Consulte el enlace en 'Conexiones a datos de mercado' para ver m\u00e1s detalles.Hay disponibles datos de mercado en diferido.SPY ARCA/TOP/ALL, contract: Option(conId=841818581, symbol='SPY', lastTradeDateOrContractMonth='20260206', strike=691.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   260206C00691000', tradingClass='SPY')
Error 10091, reqId 9: Parte de los datos de mercado solicitados requiere suscripciones adicionales para API. Consulte el enlace en 'Conexiones a datos de mercado' para ver m\u00e1s detalles.Hay disponibles datos de mercado en diferido.SPY ARCA/TOP/ALL, contract: Option(conId=841819243, symbol='SPY', lastTradeDateOrContractMonth='20260206', strike=691.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   260206P00691000', tradingClass='SPY')


Strike ATM (válido) K: 691.0000
Call localSymbol=SPY   260206C00691000 conId=841818581
Put  localSymbol=SPY   260206P00691000 conId=841819243
Call price (mid/last): nan
Put  price (mid/last): nan


: 

[AVISO] No se pudo cargar automáticamente con %run: No se pudo obtener precio de opciones (Call/Put). Probable causa: falta de permisos/suscripción de market data de OPTIONS para API.
  Las funciones compatibles se cargarán automáticamente.

[INFO] Verificando que las funciones del código original están disponibles...

  [OK] bs_price: Disponible
  [FALTA] bs_greeks_manual: FALTANTE
  [OK] implied_vol_bisect: Disponible
  [FALTA] get_risk_free_rate_yahoo: FALTANTE

  [INFO] Variables del notebook original disponibles: S, K, T, r, q, ib, df_chain, expiry

[ADVERTENCIA] Algunas funciones no están disponibles.
   Funciones faltantes: bs_greeks_manual, get_risk_free_rate_yahoo

   SOLUCIONES:
   1. Ejecuta primero todas las celdas del notebook EJERCICIO_MIAX_2025.ipynb
   2. O ejecuta este notebook en el mismo kernel donde ejecutaste el notebook original
   3. O las funciones compatibles se cargarán automáticamente en las siguientes celdas



## Funciones Compatibles

Si ejecutaste el notebook **EJERCICIO_MIAX_2025.ipynb** previamente, las siguientes variables estarán disponibles:

- **`S`**: Precio spot de SPY
- **`K`**: Strike price de la opción
- **`T`**: Tiempo hasta vencimiento (años)
- **`r`**: Tasa libre de riesgo
- **`q`**: Dividend yield
- **`sigma`**: Volatilidad implícita
- **`ib`**: Conexión a Interactive Brokers (objeto IB)
- **`df_chain`**: DataFrame con la cadena de opciones
- **`expiry`**: Fecha de vencimiento (formato YYYYMMDD)

**Si estas variables están disponibles**, se usarán automáticamente en los ejemplos.
**Si no están disponibles**, se usarán valores por defecto razonables.

**NOTA**: Las funciones compatibles de abajo se cargarán solo si no están ya disponibles del notebook original.



In [4]:
# ═══════════════════════════════════════════════════════════════════════
# FUNCIONES COMPATIBLES CON EL CÓDIGO ORIGINAL
# ═══════════════════════════════════════════════════════════════════════
# 
# Estas funciones replican la funcionalidad del código original
# usando la misma nomenclatura: S, K, T, r, q, sigma, right

import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import brentq
import math

# Verificar si las funciones ya existen (del código original)
if 'bs_price' not in dir():
    def bs_price(S: float, K: float, T: float, r: float, q: float, sigma: float, right: str) -> float:
        """
        Calcula precio Black-Scholes (compatible con código original).
        
        Args:
            S: Precio spot
            K: Strike
            T: Tiempo hasta vencimiento (años)
            r: Tasa libre de riesgo
            q: Dividend yield
            sigma: Volatilidad
            right: 'C' para Call, 'P' para Put
            
        Returns:
            Precio teórico de la opción
        """
        if T <= 0:
            return max(S - K, 0) if right == 'C' else max(K - S, 0)
        
        sqrt_T = math.sqrt(T)
        d1 = (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * sqrt_T)
        d2 = d1 - sigma * sqrt_T
        
        if right == 'C':
            price = S * math.exp(-q * T) * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
        else:  # Put
            price = K * math.exp(-r * T) * norm.cdf(-d2) - S * math.exp(-q * T) * norm.cdf(-d1)
        
        return price
    
    print(" Función bs_price() definida (compatible)")

if 'bs_greeks_manual' not in dir():
    def bs_greeks_manual(S, K, T, r, q, sigma, right):
        """
        Calcula griegas manualmente (compatible con código original).
        
        Returns:
            dict con delta, gamma, theta, vega, rho
        """
        if T <= 0:
            delta = 1.0 if (right == 'C' and S > K) or (right == 'P' and S < K) else 0.0
            return {'delta': delta, 'gamma': 0, 'theta': 0, 'vega': 0, 'rho': 0}
        
        sqrt_T = math.sqrt(T)
        d1 = (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * sqrt_T)
        d2 = d1 - sigma * sqrt_T
        
        # Delta
        if right == 'C':
            delta = math.exp(-q * T) * norm.cdf(d1)
        else:
            delta = -math.exp(-q * T) * norm.cdf(-d1)
        
        # Gamma (igual para Call y Put)
        gamma = math.exp(-q * T) * norm.pdf(d1) / (S * sigma * sqrt_T)
        
        # Theta
        theta_part1 = -S * math.exp(-q * T) * norm.pdf(d1) * sigma / (2 * sqrt_T)
        if right == 'C':
            theta_part2 = -r * K * math.exp(-r * T) * norm.cdf(d2) + q * S * math.exp(-q * T) * norm.cdf(d1)
        else:
            theta_part2 = r * K * math.exp(-r * T) * norm.cdf(-d2) - q * S * math.exp(-q * T) * norm.cdf(-d1)
        theta = (theta_part1 + theta_part2) / 365  # Por día
        
        # Vega (igual para Call y Put)
        vega = S * math.exp(-q * T) * norm.pdf(d1) * sqrt_T / 100  # Por 1% de cambio
        
        # Rho
        if right == 'C':
            rho = K * T * math.exp(-r * T) * norm.cdf(d2) / 100
        else:
            rho = -K * T * math.exp(-r * T) * norm.cdf(-d2) / 100
        
        return {'delta': delta, 'gamma': gamma, 'theta': theta, 'vega': vega, 'rho': rho}
    
    print(" Función bs_greeks_manual() definida (compatible)")

if 'implied_vol_bisect' not in dir():
    def implied_vol_bisect(price_mkt, S, K, T, r, q, right, lo=1e-4, hi=4.0, max_iter=100, tol=1e-6):
        """
        Calcula volatilidad implícita por bisección (compatible con código original).
        """
        def price_diff(sigma):
            return bs_price(S, K, T, r, q, sigma, right) - price_mkt
        
        # Verificar que hay solución
        if price_diff(lo) * price_diff(hi) > 0:
            return None
        
        # Bisección
        for _ in range(max_iter):
            mid = (lo + hi) / 2
            diff = price_diff(mid)
            
            if abs(diff) < tol:
                return mid
            
            if diff > 0:
                hi = mid
            else:
                lo = mid
        
        return (lo + hi) / 2
    
    print(" Función implied_vol_bisect() definida (compatible)")

if 'get_risk_free_rate_yahoo' not in dir():
    def get_risk_free_rate_yahoo(default: float = 0.045) -> float:
        """
        Obtiene tasa libre de riesgo (compatible con código original).
        Por defecto retorna el valor por defecto.
        """
        try:
            import yfinance as yf
            tnx = yf.Ticker("^TNX")
            rate = tnx.history(period="1d")['Close'].iloc[-1] / 100
            return rate
        except:
            return default
    
    print(" Función get_risk_free_rate_yahoo() definida (compatible)")

print("\n[OK] Todas las funciones compatibles están disponibles.")
print("   Puedes continuar con los objetivos faltantes.\n")

# Función helper: Obtener variables del notebook original

def get_original_var(var_name, default_value):
    """
    Intenta obtener una variable del notebook original EJERCICIO_MIAX_2025.ipynb.
    Si no está disponible, retorna el valor por defecto.
    
    Args:
        var_name: Nombre de la variable a buscar
        default_value: Valor por defecto si la variable no existe
        
    Returns:
        Valor de la variable original o el valor por defecto
    """
    try:
        # Intentar obtener de globals primero
        if var_name in globals():
            value = globals()[var_name]
            if value is not None:
                return value
    except:
        pass
    
    try:
        # Intentar obtener usando dir() y eval (más seguro)
        if var_name in dir():
            value = eval(var_name)
            if value is not None:
                return value
    except:
        pass
    
    # Si no se encuentra, retornar el valor por defecto
    return default_value

# Verificar y mostrar qué variables del notebook original están disponibles
print("[INFO] Variables del notebook original:")
vars_to_check = {
    'S': 'Precio spot SPY',
    'K': 'Strike price',
    'T': 'Tiempo hasta vencimiento',
    'r': 'Tasa libre de riesgo',
    'q': 'Dividend yield',
    'sigma': 'Volatilidad',
    'ib': 'Conexión Interactive Brokers',
    'df_chain': 'Cadena de opciones',
    'expiry': 'Fecha de vencimiento'
}

available_vars = []
for var, desc in vars_to_check.items():
    try:
        if var in globals() or var in dir():
            available_vars.append(var)
            print(f"  [OK] {var} ({desc}): Disponible")
    except:
        pass

if not available_vars:
    print("  [INFO] Ninguna variable del notebook original detectada.")
    print("     Se usarán valores por defecto en los ejemplos.")

print()



 Función bs_greeks_manual() definida (compatible)
 Función get_risk_free_rate_yahoo() definida (compatible)

[OK] Todas las funciones compatibles están disponibles.
   Puedes continuar con los objetivos faltantes.

[INFO] Variables del notebook original:
  [OK] S (Precio spot SPY): Disponible
  [OK] K (Strike price): Disponible
  [OK] T (Tiempo hasta vencimiento): Disponible
  [OK] r (Tasa libre de riesgo): Disponible
  [OK] q (Dividend yield): Disponible
  [OK] ib (Conexión Interactive Brokers): Disponible
  [OK] df_chain (Cadena de opciones): Disponible
  [OK] expiry (Fecha de vencimiento): Disponible



# Parte 2: Objetivos de Aprendizaje - Estrategias con Opciones

Los objetivos de esta sección se enfocan en la construcción práctica de estrategias con opciones sobre SPY, análisis de rendimiento, y consideraciones operativas y técnicas.



## 1. Construcción de Long Straddle sobre SPY

#### Explicación Teórica del Long Straddle

El **Long Straddle** es una estrategia de opciones que consiste en comprar simultáneamente una Call y una Put con el mismo strike y la misma fecha de vencimiento. Esta estrategia es rentable cuando el subyacente se mueve significativamente en cualquier dirección.

**Definición**: Compra simultánea de call y put con mismo strike y vencimiento. El objetivo es beneficiarse de movimientos grandes en cualquier dirección (alcista o bajista), sin necesidad de predecir la dirección del movimiento.

**Perfil de riesgo**: 
- Pérdida limitada: La máxima pérdida es igual a la prima total pagada (precio de la call + precio de la put)
- Beneficio ilimitado: Si el precio se mueve lo suficiente en cualquier dirección, el beneficio puede ser ilimitado
- Puntos de equilibrio: Strike ± Prima total

**Cuándo usar**: Esta estrategia es ideal cuando se espera alta volatilidad sin una dirección clara del mercado. Es especialmente útil antes de eventos importantes (anuncios de resultados, decisiones de política monetaria, etc.) donde se anticipa un movimiento significativo pero la dirección es incierta.

#### Justificación de Parámetros ATM (At-The-Money)

Usar strikes ATM en lugar de otras configuraciones ofrece varias ventajas clave:

**Máxima sensibilidad a Gamma**: Las opciones ATM tienen el gamma más alto, lo que significa que maximizan los beneficios de movimientos grandes del subyacente. Cuando el precio se mueve, el delta cambia más rápidamente en opciones ATM, amplificando las ganancias.

**Simetría perfecta**: Un straddle ATM ofrece igual potencial de beneficio al alza y a la baja. Esto es crucial porque no necesitamos predecir la dirección, solo necesitamos movimiento.

**Mayor Vega**: Las opciones ATM son las más sensibles a cambios en volatilidad implícita. Si la volatilidad aumenta después de abrir la posición, el valor del straddle aumenta significativamente.

**Theta más negativo**: El decay temporal es máximo en ATM, lo cual representa el coste principal de mantener la estrategia. Sin embargo, este coste se compensa con la mayor sensibilidad a movimientos.

**Comparación con OTM (Out-The-Money)**: Aunque las opciones OTM tienen menor coste inicial, necesitan movimientos mucho mayores para ser rentables. El straddle OTM requiere que el precio se mueva más allá de ambos strikes OTM, reduciendo la probabilidad de éxito.

**Comparación con ITM (In-The-Money)**: Las opciones ITM tienen mayor coste y menos apalancamiento. Además, parte del valor es intrínseco, lo que reduce la sensibilidad a cambios de volatilidad.

#### Selección de Vencimiento

La elección del vencimiento es crucial y depende de varios factores:

**Horizonte temporal de la estrategia**:
- Corto plazo (semanal, 7-14 días): Mayor Theta decay, requiere movimientos más rápidos
- Medio plazo (mensual, 30-45 días): Balance óptimo entre coste y sensibilidad
- Largo plazo (trimestral, 60-90 días): Menor Theta decay pero mayor coste inicial

**Trade-off entre Theta y Vega**:
- Vencimientos cortos: Mayor Theta decay (más caro mantener día a día), menor Vega (menos sensible a cambios de IV)
- Vencimientos largos: Menor Theta decay (menos coste diario), mayor Vega (más sensible a cambios de IV)

**Liquidez**: Los vencimientos más cercanos suelen tener mayor volumen y spreads más estrechos, reduciendo costos de ejecución.

**Recomendación**: Usar vencimientos entre 30-45 días para balance óptimo entre coste de mantenimiento (Theta) y sensibilidad a volatilidad (Vega), además de buena liquidez.



### 1.1. Función Principal: construir_long_straddle

Esta función construye un Long Straddle obteniendo datos del broker (si están disponibles) o usando datos del notebook EJERCICIO_MIAX_2025 si ya fueron obtenidos. La función:
- Obtiene el precio actual del subyacente desde el broker o datos previos
- Identifica el strike ATM más cercano consultando la cadena de opciones
- Calcula precios y griegas usando nuestras funciones de Black-Scholes
- Retorna toda la información necesaria para analizar la estrategia


In [ ]:
# Implementación: Función construir_long_straddle

def construir_long_straddle(ticker='SPY', dias_vencimiento=30, fecha_analisis=None):
    """
    Construye un Long Straddle sobre el ticker especificado.
    
    Esta función obtiene el precio actual del subyacente, identifica el strike ATM,
    selecciona las opciones Call y Put correspondientes, calcula precios y griegas,
    y retorna toda la información necesaria para analizar la estrategia.
    
    Args:
        ticker: Símbolo del activo (por defecto 'SPY')
        dias_vencimiento: Días hasta el vencimiento deseado (aproximado)
        fecha_analisis: Fecha en la que se construye el straddle (por defecto hoy)
    
    Returns:
        Diccionario con toda la información del straddle construido
    """
    from datetime import datetime, timedelta
    
    # Si no se especifica fecha, usar hoy
    if fecha_analisis is None:
        fecha_analisis = datetime.now()
    
    # Obtener precio actual de SPY
    # En un entorno real, esto usaría las funciones del notebook anterior para obtener precio en tiempo real
    # Por ahora, simulamos obteniendo un precio base
    try:
        # Intentar usar precio del código original si existe
        precio_subyacente = S if 'S' in dir() else 450.0
    except:
        precio_subyacente = 450.0
    
    # Calcular fecha de vencimiento aproximada
    # En un entorno real, buscaríamos el vencimiento más cercano a dias_vencimiento
    vencimiento = fecha_analisis + timedelta(days=dias_vencimiento)
    dias_hasta_vencimiento = (vencimiento - fecha_analisis).days
    T = dias_hasta_vencimiento / 365.0
    
    # Identificar strike ATM: el strike más cercano al precio actual
    # En un entorno real, esto se haría consultando la cadena de opciones
    # Por ahora, redondeamos al strike más cercano (espaciado típico de $5 para SPY)
    strike_spacing = 5.0
    strike = round(precio_subyacente / strike_spacing) * strike_spacing
    
    # Obtener parámetros de mercado (reutilizando variables del código original)
    try:
        r = r if 'r' in dir() else 0.05
        q = q if 'q' in dir() else 0.0
        sigma = sigma if 'sigma' in dir() else 0.15
    except:
        r = 0.05
        q = 0.0
        sigma = 0.15
    
    # Calcular precios de Call y Put usando bs_price del notebook anterior
    # REUTILIZAMOS la función bs_price del código original
    precio_call = bs_price(precio_subyacente, strike, T, r, q, sigma, 'C')
    precio_put = bs_price(precio_subyacente, strike, T, r, q, sigma, 'P')
    
    # En un entorno real, también obtendríamos bid/ask del mercado
    # Por ahora, usamos el precio teórico como mid
    precio_call_bid = precio_call * 0.99  # Simulación: bid ligeramente menor
    precio_call_ask = precio_call * 1.01  # Simulación: ask ligeramente mayor
    precio_call_mid = precio_call
    
    precio_put_bid = precio_put * 0.99
    precio_put_ask = precio_put * 1.01
    precio_put_mid = precio_put
    
    # Calcular griegas individuales usando bs_greeks_manual del notebook anterior
    # REUTILIZAMOS la función bs_greeks_manual del código original
    griegas_call = bs_greeks_manual(precio_subyacente, strike, T, r, q, sigma, 'C')
    griegas_put = bs_greeks_manual(precio_subyacente, strike, T, r, q, sigma, 'P')
    
    # Calcular griegas totales del straddle
    # Delta total: Call + Put (debería estar cerca de 0 si es verdadero ATM)
    delta_total = griegas_call['delta'] + griegas_put['delta']
    
    # Gamma total: suma de gammas (siempre positivo)
    gamma_total = griegas_call['gamma'] + griegas_put['gamma']
    
    # Theta total: suma de thetas (siempre negativo, costo temporal)
    theta_total = griegas_call['theta'] + griegas_put['theta']
    
    # Vega total: suma de vegas (siempre positivo, beneficio de aumento de IV)
    vega_total = griegas_call['vega'] + griegas_put['vega']
    
    # Rho total: suma de rhos (normalmente cerca de 0, efectos opuestos)
    rho_total = griegas_call['rho'] + griegas_put['rho']
    
    # Calcular inversión inicial (prima total pagada)
    inversion_inicial = precio_call_mid + precio_put_mid
    
    # Calcular puntos de equilibrio
    breakeven_superior = strike + inversion_inicial
    breakeven_inferior = strike - inversion_inicial
    rango_beneficio = breakeven_superior - breakeven_inferior
    
    # Información de contratos (simulada, en entorno real vendría del broker)
    call_contract = {
        'symbol': f'{ticker}',
        'strike': strike,
        'expiry': vencimiento,
        'right': 'C',
        'bid': precio_call_bid,
        'ask': precio_call_ask,
        'mid': precio_call_mid
    }
    
    put_contract = {
        'symbol': f'{ticker}',
        'strike': strike,
        'expiry': vencimiento,
        'right': 'P',
        'bid': precio_put_bid,
        'ask': precio_put_ask,
        'mid': precio_put_mid
    }
    
    # Retornar diccionario con toda la información
    return {
        'fecha': fecha_analisis,
        'precio_subyacente': precio_subyacente,
        'vencimiento': vencimiento,
        'dias_hasta_vencimiento': dias_hasta_vencimiento,
        'strike': strike,
        'call_contract': call_contract,
        'put_contract': put_contract,
        'precio_call': precio_call_mid,
        'precio_put': precio_put_mid,
        'inversion_inicial': inversion_inicial,
        'griegas_call': griegas_call,
        'griegas_put': griegas_put,
        'griegas_totales': {
            'delta': delta_total,
            'gamma': gamma_total,
            'theta': theta_total,
            'vega': vega_total,
            'rho': rho_total
        },
        'breakeven_superior': breakeven_superior,
        'breakeven_inferior': breakeven_inferior,
        'rango_beneficio': rango_beneficio
    }


### 1.2. Ejemplo de Uso

Esta celda demuestra cómo usar la función `construir_long_straddle` y muestra los resultados de forma estructurada, incluyendo una tabla resumen con todas las métricas clave del straddle construido.


In [ ]:
# Ejemplo de uso de construir_long_straddle

print(f"\n{'='*60}")
print(f"EJEMPLO DE USO: CONSTRUCCIÓN DE LONG STRADDLE")
print(f"{'='*60}\n")

# Llamar a la función con parámetros ejemplo
straddle_info = construir_long_straddle(ticker='SPY', dias_vencimiento=30)

# Mostrar resultados de forma clara y estructurada
print(f"INFORMACIÓN DEL STRADDLE CONSTRUIDO:")
print(f"{'='*60}")
print(f"Fecha de análisis: {straddle_info['fecha']}")
print(f"Precio de SPY: ${straddle_info['precio_subyacente']:.2f}")
print(f"Strike elegido: ${straddle_info['strike']:.2f}")
print(f"Vencimiento: {straddle_info['vencimiento']}")
print(f"Días hasta vencimiento: {straddle_info['dias_hasta_vencimiento']} días")
print(f"\nPRECIOS DE OPCIONES:")
print(f"  Call: ${straddle_info['precio_call']:.2f}")
print(f"  Put: ${straddle_info['precio_put']:.2f}")
print(f"  Inversión total: ${straddle_info['inversion_inicial']:.2f}")
print(f"\nGRIEGAS TOTALES DEL STRADDLE:")
griegas = straddle_info['griegas_totales']
print(f"  Delta: {griegas['delta']:.4f} (cerca de 0 = neutral direccional)")
print(f"  Gamma: {griegas['gamma']:.6f} (positivo = beneficio de movimientos)")
print(f"  Theta: ${griegas['theta']:.2f}/día (negativo = costo temporal)")
print(f"  Vega: ${griegas['vega']:.2f} (positivo = beneficio de aumento de IV)")
print(f"  Rho: ${griegas['rho']:.2f} (cerca de 0 = poco sensible a tasas)")
print(f"\nPUNTOS DE EQUILIBRIO:")
print(f"  Breakeven superior: ${straddle_info['breakeven_superior']:.2f}")
print(f"  Breakeven inferior: ${straddle_info['breakeven_inferior']:.2f}")
print(f"  Rango de beneficio: ${straddle_info['rango_beneficio']:.2f}")
print(f"{'='*60}\n")

# Crear tabla con información clave usando pandas
import pandas as pd

tabla_info = pd.DataFrame({
    'Métrica': [
        'Precio SPY',
        'Strike',
        'Vencimiento (días)',
        'Precio Call',
        'Precio Put',
        'Inversión Total',
        'Delta Total',
        'Gamma Total',
        'Theta Total ($/día)',
        'Vega Total',
        'Rho Total',
        'Breakeven Superior',
        'Breakeven Inferior'
    ],
    'Valor': [
        f"${straddle_info['precio_subyacente']:.2f}",
        f"${straddle_info['strike']:.2f}",
        f"{straddle_info['dias_hasta_vencimiento']}",
        f"${straddle_info['precio_call']:.2f}",
        f"${straddle_info['precio_put']:.2f}",
        f"${straddle_info['inversion_inicial']:.2f}",
        f"{griegas['delta']:.4f}",
        f"{griegas['gamma']:.6f}",
        f"${griegas['theta']:.2f}",
        f"${griegas['vega']:.2f}",
        f"${griegas['rho']:.2f}",
        f"${straddle_info['breakeven_superior']:.2f}",
        f"${straddle_info['breakeven_inferior']:.2f}"
    ]
})

print("TABLA RESUMEN:")
print(tabla_info.to_string(index=False))
print()


#### Nota sobre Backtesting

El código de backtesting con la clase `LongStraddleStrategy` se implementa más adelante en el Objetivo 2.1. Por ahora, hemos completado la construcción básica del Long Straddle con la función `construir_long_straddle`.


In [ ]:
# Clase para backtesting (se usará más adelante en Objetivo 2.1)
class LongStraddleStrategy:
    """
    Estrategia Long Straddle para backtesting.
    
    REUTILIZA bs_price() del código original.
    """
    
    def __init__(self, entry_frequency_days=7, exit_at_expiry=True, 
                 stop_loss_pct=0.50, take_profit_pct=1.0):
        self.entry_frequency = entry_frequency_days
        self.exit_at_expiry = exit_at_expiry
        self.stop_loss = stop_loss_pct
        self.take_profit = take_profit_pct
        self.trades = []
    
    def backtest(self, S_prices, dates, r, q, sigma, strike_spacing=5.0):
        """
        Backtest de la estrategia.
        
        Args (usando nomenclatura original):
            S_prices: Precios históricos de SPY (variable S)
            dates: Fechas correspondientes
            r: Tasa libre de riesgo (variable r)
            q: Dividend yield (variable q)
            sigma: Volatilidad (variable sigma)
            strike_spacing: Espaciado de strikes
        """
        self.trades = []
        current_date_idx = 0
        
        while current_date_idx < len(dates) - 30:
            entry_date_idx = current_date_idx
            S_entry = S_prices[entry_date_idx]
            entry_date = dates[entry_date_idx]
            
            # Strike ATM (redondeado)
            K_strike = round(S_entry / strike_spacing) * strike_spacing
            
            # Calcular precio de Call y Put (30 días) - REUTILIZAMOS bs_price
            T = 30 / 365.0
            call_price = bs_price(S_entry, K_strike, T, r, q, sigma, 'C')
            put_price = bs_price(S_entry, K_strike, T, r, q, sigma, 'P')
            total_cost = call_price + put_price
            
            # Simular hasta vencimiento o salida temprana
            exit_date_idx = min(entry_date_idx + 30, len(dates) - 1)
            exit_reason = 'expiry'
            exit_pnl = None
            
            for day_idx in range(entry_date_idx + 1, exit_date_idx + 1):
                S_current = S_prices[day_idx]
                days_left = 30 - (day_idx - entry_date_idx)
                T_left = days_left / 365.0
                
                if T_left > 0:
                    # REUTILIZAMOS bs_price del código original
                    current_call = bs_price(S_current, K_strike, T_left, r, q, sigma, 'C')
                    current_put = bs_price(S_current, K_strike, T_left, r, q, sigma, 'P')
                    current_value = current_call + current_put
                else:
                    current_call = max(S_current - K_strike, 0)
                    current_put = max(K_strike - S_current, 0)
                    current_value = current_call + current_put
                
                pnl = current_value - total_cost
                pnl_pct = pnl / total_cost
                
                if pnl_pct <= -self.stop_loss:
                    exit_date_idx = day_idx
                    exit_reason = 'stop_loss'
                    exit_pnl = pnl
                    break
                elif pnl_pct >= self.take_profit:
                    exit_date_idx = day_idx
                    exit_reason = 'take_profit'
                    exit_pnl = pnl
                    break
            
            if exit_pnl is None:
                S_final = S_prices[exit_date_idx]
                final_call = max(S_final - K_strike, 0)
                final_put = max(K_strike - S_final, 0)
                exit_pnl = (final_call + final_put) - total_cost
            
            self.trades.append({
                'entry_date': entry_date,
                'exit_date': dates[exit_date_idx],
                'S_entry': S_entry,
                'S_exit': S_prices[exit_date_idx],
                'K': K_strike,
                'cost': total_cost,
                'pnl': exit_pnl,
                'pnl_pct': exit_pnl / total_cost,
                'exit_reason': exit_reason,
                'days_held': exit_date_idx - entry_date_idx
            })
            
            current_date_idx = entry_date_idx + self.entry_frequency
        
        return pd.DataFrame(self.trades)
    
    def calculate_metrics(self, trades_df):
        """Calcula métricas de performance."""
        if len(trades_df) == 0:
            return {}
        
        total_return = trades_df['pnl'].sum()
        win_rate = (trades_df['pnl'] > 0).sum() / len(trades_df)
        avg_win = trades_df[trades_df['pnl'] > 0]['pnl'].mean() if (trades_df['pnl'] > 0).sum() > 0 else 0
        avg_loss = trades_df[trades_df['pnl'] < 0]['pnl'].mean() if (trades_df['pnl'] < 0).sum() > 0 else 0
        
        equity_curve = trades_df['pnl'].cumsum()
        max_equity = equity_curve.cummax()
        drawdown = equity_curve - max_equity
        max_drawdown = drawdown.min()
        
        returns = trades_df['pnl_pct']
        sharpe = returns.mean() / returns.std() * np.sqrt(252 / self.entry_frequency) if returns.std() > 0 else 0
        
        gross_profit = trades_df[trades_df['pnl'] > 0]['pnl'].sum() if (trades_df['pnl'] > 0).sum() > 0 else 0
        gross_loss = abs(trades_df[trades_df['pnl'] < 0]['pnl'].sum()) if (trades_df['pnl'] < 0).sum() > 0 else 1
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else 0
        
        return {
            'total_return': total_return,
            'num_trades': len(trades_df),
            'win_rate': win_rate,
            'avg_win': avg_win,
            'avg_loss': avg_loss,
            'max_drawdown': max_drawdown,
            'sharpe_ratio': sharpe,
            'profit_factor': profit_factor
        }


### 1.3. Visualización del Payoff

Esta celda crea un gráfico del payoff teórico del Long Straddle, mostrando:
- La forma característica del payoff (ganancia ilimitada en ambas direcciones)
- Los puntos de equilibrio (breakeven superior e inferior)
- Las zonas de ganancia y pérdida sombreadas
- El precio actual y el strike elegido


In [ ]:
# Visualización inicial del Payoff del Straddle

# Imports necesarios
import numpy as np
import matplotlib.pyplot as plt

# Obtener información del straddle construido
straddle_info = construir_long_straddle(ticker='SPY', dias_vencimiento=30)

# Preparar datos para el gráfico
precio_actual = straddle_info['precio_subyacente']
strike = straddle_info['strike']
prima_total = straddle_info['inversion_inicial']
breakeven_sup = straddle_info['breakeven_superior']
breakeven_inf = straddle_info['breakeven_inferior']

# Crear rango de precios de SPY al vencimiento (desde -20% hasta +20% del precio actual)
rango_porcentaje = 0.20
precios_vencimiento = np.linspace(
    precio_actual * (1 - rango_porcentaje), 
    precio_actual * (1 + rango_porcentaje), 
    200
)

# Calcular payoff teórico del straddle: max(S-K, 0) + max(K-S, 0) - Prima_total
payoff_straddle = np.maximum(precios_vencimiento - strike, 0) + \
                  np.maximum(strike - precios_vencimiento, 0) - \
                  prima_total

# Crear gráfico
fig, ax = plt.subplots(figsize=(14, 8))

# Dibujar línea del payoff
ax.plot(precios_vencimiento, payoff_straddle, 'b-', linewidth=2.5, label='Payoff del Straddle')

# Marcar líneas verticales importantes
ax.axvline(precio_actual, color='green', linestyle='--', linewidth=2, alpha=0.7, label=f'Precio actual: ${precio_actual:.2f}')
ax.axvline(strike, color='red', linestyle='--', linewidth=2, alpha=0.7, label=f'Strike: ${strike:.2f}')
ax.axvline(breakeven_sup, color='orange', linestyle='--', linewidth=2, alpha=0.7, label=f'Breakeven superior: ${breakeven_sup:.2f}')
ax.axvline(breakeven_inf, color='orange', linestyle='--', linewidth=2, alpha=0.7, label=f'Breakeven inferior: ${breakeven_inf:.2f}')

# Línea horizontal en cero
ax.axhline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)

# Sombrear zona de pérdida (entre breakevens)
zona_perdida = (precios_vencimiento >= breakeven_inf) & (precios_vencimiento <= breakeven_sup)
ax.fill_between(precios_vencimiento, payoff_straddle, 0, where=zona_perdida, 
                color='red', alpha=0.2, label='Zona de pérdida')

# Sombrear zona de ganancia (fuera de breakevens)
zona_ganancia = (precios_vencimiento < breakeven_inf) | (precios_vencimiento > breakeven_sup)
ax.fill_between(precios_vencimiento, payoff_straddle, 0, where=zona_ganancia, 
                color='green', alpha=0.2, label='Zona de ganancia')

# Configuración del gráfico
ax.set_xlabel('Precio de SPY al Vencimiento ($)', fontsize=12, fontweight='bold')
ax.set_ylabel('Profit/Loss ($)', fontsize=12, fontweight='bold')
ax.set_title('Payoff Diagram: Long Straddle sobre SPY', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xlim(precios_vencimiento[0], precios_vencimiento[-1])

plt.tight_layout()
plt.savefig('images/straddle_payoff.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Gráfico de payoff guardado")

### 1.4. Análisis de Sensibilidad a Diferentes Strikes

Esta celda analiza cómo cambian las características del straddle al variar el strike desde ATM-2% hasta ATM+2%. El análisis incluye:
- Comparación de primas totales para diferentes strikes
- Evolución de las griegas (Delta, Gamma, Theta, Vega)
- Impacto en el rango de beneficio (distancia entre breakevens)
- Visualizaciones comparativas que muestran el trade-off entre diferentes configuraciones


In [ ]:
# Análisis de sensibilidad a diferentes strikes

# Imports necesarios
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Obtener precio base y parámetros
try:
    precio_base = S if 'S' in dir() else 450.0
    r_base = r if 'r' in dir() else 0.05
    q_base = q if 'q' in dir() else 0.0
    sigma_base = sigma if 'sigma' in dir() else 0.15
except:
    precio_base = 450.0
    r_base = 0.05
    q_base = 0.0
    sigma_base = 0.15

T = 30 / 365.0
strike_spacing = 5.0

# Definir strikes a analizar (desde ATM-2% hasta ATM+2%)
strikes_analisis = []
porcentajes = [-0.02, -0.01, 0.0, 0.01, 0.02]
nombres = ['ATM-2%', 'ATM-1%', 'ATM', 'ATM+1%', 'ATM+2%']

for pct in porcentajes:
    strike = round((precio_base * (1 + pct)) / strike_spacing) * strike_spacing
    strikes_analisis.append(strike)

# Calcular métricas para cada strike
resultados_sensibilidad = []

for i, strike in enumerate(strikes_analisis):
    # Calcular precios de Call y Put usando bs_price
    precio_call = bs_price(precio_base, strike, T, r_base, q_base, sigma_base, 'C')
    precio_put = bs_price(precio_base, strike, T, r_base, q_base, sigma_base, 'P')
    prima_total = precio_call + precio_put
    
    # Calcular griegas usando bs_greeks_manual
    griegas_call = bs_greeks_manual(precio_base, strike, T, r_base, q_base, sigma_base, 'C')
    griegas_put = bs_greeks_manual(precio_base, strike, T, r_base, q_base, sigma_base, 'P')
    
    delta_total = griegas_call['delta'] + griegas_put['delta']
    gamma_total = griegas_call['gamma'] + griegas_put['gamma']
    theta_total = griegas_call['theta'] + griegas_put['theta']
    vega_total = griegas_call['vega'] + griegas_put['vega']
    
    # Calcular breakevens
    breakeven_sup = strike + prima_total
    breakeven_inf = strike - prima_total
    rango_beneficio = breakeven_sup - breakeven_inf
    
    resultados_sensibilidad.append({
        'Strike': nombres[i],
        'Strike_Valor': strike,
        'Prima_Total': prima_total,
        'Delta_Total': delta_total,
        'Gamma_Total': gamma_total,
        'Theta_Total': theta_total,
        'Vega_Total': vega_total,
        'Breakeven_Superior': breakeven_sup,
        'Breakeven_Inferior': breakeven_inf,
        'Rango_Beneficio': rango_beneficio
    })

# Crear DataFrame y mostrar tabla
df_sensibilidad = pd.DataFrame(resultados_sensibilidad)

print(f"\n{'='*60}")
print(f"ANÁLISIS DE SENSIBILIDAD A DIFERENTES STRIKES")
print(f"{'='*60}")
print(f"Precio base de SPY: ${precio_base:.2f}")
print(f"Vencimiento: {int(T*365)} días")
print(f"{'='*60}\n")

# Formatear tabla para mejor visualización
df_display = pd.DataFrame({
    'Strike': df_sensibilidad['Strike'],
    'Strike ($)': df_sensibilidad['Strike_Valor'].apply(lambda x: f"${x:.2f}"),
    'Prima Total ($)': df_sensibilidad['Prima_Total'].apply(lambda x: f"${x:.2f}"),
    'Delta Total': df_sensibilidad['Delta_Total'].apply(lambda x: f"{x:.4f}"),
    'Gamma Total': df_sensibilidad['Gamma_Total'].apply(lambda x: f"{x:.6f}"),
    'Theta Total ($/día)': df_sensibilidad['Theta_Total'].apply(lambda x: f"${x:.2f}"),
    'Vega Total': df_sensibilidad['Vega_Total'].apply(lambda x: f"${x:.2f}"),
    'Rango Beneficio ($)': df_sensibilidad['Rango_Beneficio'].apply(lambda x: f"${x:.2f}")
})

print(df_display.to_string(index=False))
print(f"\n{'='*60}")
print("INTERPRETACIÓN:")
print(f"{'='*60}")
print("ATM-2%: Ligeramente OTM Put, ITM Call - Menor prima pero necesita movimiento mayor")
print("ATM-1%: Casi ATM - Similar a ATM pero con ligera asimetría")
print("ATM: Strike perfecto - Máxima simetría y sensibilidad a movimientos")
print("ATM+1%: Casi ATM - Similar a ATM pero con ligera asimetría")
print("ATM+2%: Ligeramente ITM Put, OTM Call - Mayor prima pero menos apalancamiento")
print(f"{'='*60}\n")

# Visualización comparativa
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Prima Total vs Strike
ax = axes[0, 0]
ax.plot(df_sensibilidad['Strike_Valor'], df_sensibilidad['Prima_Total'], 'o-', linewidth=2, markersize=8)
ax.axvline(precio_base, color='r', linestyle='--', alpha=0.5, label='Precio actual')
ax.set_xlabel('Strike ($)', fontsize=11)
ax.set_ylabel('Prima Total ($)', fontsize=11)
ax.set_title('Prima Total vs Strike', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Gráfico 2: Delta Total vs Strike
ax = axes[0, 1]
ax.plot(df_sensibilidad['Strike_Valor'], df_sensibilidad['Delta_Total'], 'o-', linewidth=2, markersize=8, color='green')
ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.axvline(precio_base, color='r', linestyle='--', alpha=0.5, label='Precio actual')
ax.set_xlabel('Strike ($)', fontsize=11)
ax.set_ylabel('Delta Total', fontsize=11)
ax.set_title('Delta Total vs Strike (debe estar cerca de 0 en ATM)', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Gráfico 3: Gamma Total vs Strike
ax = axes[1, 0]
ax.plot(df_sensibilidad['Strike_Valor'], df_sensibilidad['Gamma_Total'], 'o-', linewidth=2, markersize=8, color='orange')
ax.axvline(precio_base, color='r', linestyle='--', alpha=0.5, label='Precio actual')
ax.set_xlabel('Strike ($)', fontsize=11)
ax.set_ylabel('Gamma Total', fontsize=11)
ax.set_title('Gamma Total vs Strike (máximo en ATM)', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Gráfico 4: Rango de Beneficio vs Strike
ax = axes[1, 1]
ax.plot(df_sensibilidad['Strike_Valor'], df_sensibilidad['Rango_Beneficio'], 'o-', linewidth=2, markersize=8, color='purple')
ax.axvline(precio_base, color='r', linestyle='--', alpha=0.5, label='Precio actual')
ax.set_xlabel('Strike ($)', fontsize=11)
ax.set_ylabel('Rango de Beneficio ($)', fontsize=11)
ax.set_title('Distancia entre Breakevens vs Strike', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/straddle_sensibilidad_strikes.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Gráficos de sensibilidad guardados")

## 2. Versión Delta-Hedged del Straddle

#### Explicación Teórica del Delta-Hedging

El **Delta-Hedging** es una técnica de gestión de riesgo que neutraliza la exposición direccional de una posición de opciones mediante la compra o venta del activo subyacente. En el contexto de un Long Straddle, el delta-hedging transforma la estrategia de una apuesta direccional (que gana con movimientos grandes en cualquier dirección) en una apuesta pura sobre volatilidad.

**Definición**: El delta-hedging consiste en mantener una posición en el subyacente (SPY en nuestro caso) que neutralice el delta total del straddle. Como el delta cambia con el precio del subyacente (debido al gamma), es necesario rebalancear periódicamente la posición de cobertura.

#### Decisión de usar SPY como instrumento de cobertura

**Por qué usar SPY (el ETF) para cubrir:**

**Ventajas:**
- **Accesibilidad**: SPY es directamente negociable, alta liquidez
- **Simplicidad**: no requiere cuenta de futuros ni conocimiento especializado
- **Granularidad**: podemos ajustar con precisión (lotes de 100 acciones)
- **Costes**: spreads muy estrechos (típicamente 1 cent)
- **Horario**: opera en horario de mercado regular (9:30-16:00 ET)
- **Dividendos**: recibimos dividendos si mantenemos posición larga

**Desventajas:**
- **Capital**: requiere capital para comprar/vender acciones (vs futuros que son apalancados)
- **Overnight risk**: exposición fuera del horario de mercado
- **Dividendos**: complican el cálculo de delta teórico (opciones no pagan dividendos)
- **Menos eficiente para posiciones muy grandes**

**Alternativas no elegidas:**
- **Futuros ES (E-mini S&P 500)**:
  - Ventaja: apalancamiento, trading casi 24h, no requiere capital completo
  - Desventaja: tamaño del contrato (50 × índice SPX), complejidad para retail, comisiones diferentes
- **Opciones adicionales**:
  - Ventaja: mantienes exposición a gamma/vega
  - Desventaja: mucho más complejo, afecta otras griegas

**Conclusión**: Para este ejercicio académico, SPY es la elección más directa y didáctica.

#### Justificación de la frecuencia de rebalanceo diario

**Por qué diario:**
- Balance entre gestión activa y practicidad
- El delta de un straddle cambia significativamente con movimientos del 1-2% (típico diario)
- Evita acumulación excesiva de exposición direccional
- Permite observar el patrón de "gamma scalping" claramente
- Frecuencia estándar en la industria para volatility trading

**Comparación con otras frecuencias:**

**Rebalanceo intradiario (cada hora/minuto):**
- Ventaja: delta casi siempre cerca de cero, menos riesgo direccional
- Desventaja: costes de transacción prohibitivos, overhedging, impacto de spreads
- Cuándo se usa: market makers profesionales con costes muy bajos

**Rebalanceo basado en umbrales (ej: cuando |delta| > 0.3):**
- Ventaja: reduces número de transacciones, solo rebalanceas cuando es necesario
- Desventaja: en mercados volátiles puedes pasar mucho tiempo sin cobertura
- Cuándo se usa: para reducir costes manteniendo control de riesgo

**Rebalanceo semanal/sin rebalanceo:**
- Ventaja: mínimos costes de transacción
- Desventaja: acumulas exposición direccional grande, el straddle se convierte en apuesta direccional
- Cuándo se usa: cuando NO quieres hedgear (apuestas por volatilidad + dirección)

**Sin costes de transacción en nuestra simulación:** Aunque no los simulamos explícitamente, es importante reconocer que en la realidad:
- Cada rebalanceo tiene coste: comisiones + spread bid-ask
- Rebalanceo diario con SPY: ~0.01 USD × número_acciones × 2 (compra/venta)
- Para 1 straddle (delta ~0.5 inicial) necesitas ~50 acciones → coste ~1 USD por rebalanceo
- En 30 días: ~30 USD de costes vs prima del straddle de ~1500 USD → 2% del capital
- Este coste debe compararse con el beneficio del gamma scalping


### 2.1. Función Principal: aplicar_delta_hedge

Esta función implementa el delta-hedging diario de un Long Straddle usando SPY como instrumento de cobertura. La función:
- Calcula el delta del straddle en cada día usando nuestras funciones de griegas
- Determina cuántas acciones de SPY necesitamos para neutralizar el delta
- Ejecuta rebalanceos diarios registrando todas las operaciones
- Calcula P&L diario del straddle y del hedge
- Retorna un historial completo de rebalanceos y métricas de resumen

In [ ]:
# Implementación: Función aplicar_delta_hedge

def aplicar_delta_hedge(info_straddle, precios_historicos_spy, frecuencia='diaria'):
    """
    Aplica delta-hedging diario a un Long Straddle usando SPY como instrumento de cobertura.
    
    CONCEPTO CLAVE: Delta-hedging convierte el straddle de una apuesta direccional 
    en una apuesta pura de volatilidad
    - Al neutralizar delta, eliminamos exposición a movimientos pequeños del subyacente
    - PERO mantenemos exposición positiva a Gamma (beneficio de movimientos grandes)
    - TAMBIÉN mantenemos exposición a Vega (beneficio de aumentos de IV)
    - Y mantenemos Theta negativo (coste de mantener la posición)
    
    GAMMA SCALPING: El patrón de comprar/vender acciones mientras rebalanceamos
    Si SPY sube: el delta del straddle se hace más positivo (el call gana delta)
      → vendemos acciones para neutralizar (vendemos caro)
    Si SPY baja: el delta del straddle se hace más negativo (el put gana delta absoluto)
      → compramos acciones para neutralizar (compramos barato)
    Este "comprar barato, vender caro" sistemáticamente es el gamma scalping
    En mercados volátiles (muchos movimientos): gamma scalping > theta decay → ganancia
    En mercados tranquilos (pocos movimientos): theta decay > gamma scalping → pérdida
    
    Args:
        info_straddle: Diccionario retornado por construir_long_straddle con:
            - fecha: fecha de construcción del straddle
            - precio_subyacente: precio inicial de SPY
            - strike: strike del straddle
            - vencimiento: fecha de vencimiento
            - dias_hasta_vencimiento: días hasta vencimiento inicial
            - precio_call: precio inicial del call
            - precio_put: precio inicial del put
            - inversion_inicial: prima total pagada
        precios_historicos_spy: DataFrame con precios históricos de SPY
            Columnas requeridas: fecha (o índice datetime), close (precio de cierre)
            Debe contener datos desde fecha del straddle hasta vencimiento
        frecuencia: 'diaria' (por defecto), extensible a 'hora', 'umbral', etc.
    
    Returns:
        Diccionario con:
            - historial_rebalanceos: DataFrame con todos los registros diarios
            - metricas_resumen: diccionario con métricas agregadas
    """
    import pandas as pd
    import numpy as np
    from datetime import datetime, timedelta
    
    # Obtener parámetros del straddle
    fecha_inicial = info_straddle['fecha']
    precio_inicial_spy = info_straddle['precio_subyacente']
    strike = info_straddle['strike']
    vencimiento = info_straddle['vencimiento']
    dias_hasta_vencimiento_inicial = info_straddle['dias_hasta_vencimiento']
    inversion_inicial = info_straddle['inversion_inicial']
    
    # Obtener parámetros de mercado (reutilizando variables del código original)
    # Intentar obtener de variables globales, si no usar valores por defecto
    try:
        import __main__
        r = getattr(__main__, 'r', 0.05)
        q = getattr(__main__, 'q', 0.0)
        sigma = getattr(__main__, 'sigma', 0.15)
    except:
        r = 0.05
        q = 0.0
        sigma = 0.15
    
    # Preparar datos históricos de SPY
    # Asegurar que tenemos columna de fecha y close
    if isinstance(precios_historicos_spy.index, pd.DatetimeIndex):
        df_spy = precios_historicos_spy.copy()
        df_spy['fecha'] = df_spy.index
    else:
        df_spy = precios_historicos_spy.copy()
        if 'fecha' not in df_spy.columns:
            df_spy['fecha'] = pd.to_datetime(df_spy.index)
    
    # Asegurar columna close
    if 'close' not in df_spy.columns:
        if 'Close' in df_spy.columns:
            df_spy['close'] = df_spy['Close']
        else:
            raise ValueError("DataFrame debe tener columna 'close' o 'Close'")
    
    # Filtrar datos desde fecha inicial hasta vencimiento
    df_spy['fecha'] = pd.to_datetime(df_spy['fecha'])
    fecha_inicial_dt = pd.to_datetime(fecha_inicial)
    vencimiento_dt = pd.to_datetime(vencimiento)
    
    df_spy = df_spy[(df_spy['fecha'] >= fecha_inicial_dt) & (df_spy['fecha'] <= vencimiento_dt)]
    df_spy = df_spy.sort_values('fecha').reset_index(drop=True)
    
    if len(df_spy) == 0:
        raise ValueError(f"No hay datos históricos de SPY entre {fecha_inicial_dt} y {vencimiento_dt}")
    
    # Inicialización
    posicion_spy = 0  # Empezamos sin hedge
    posicion_spy_anterior = 0  # Inicializar para primera iteración
    historial = []
    
    # Valor inicial del straddle
    valor_straddle_anterior = inversion_inicial
    precio_spy_anterior = precio_inicial_spy
    
    # Para cada día de trading desde construcción hasta vencimiento
    for idx, row in df_spy.iterrows():
        fecha_actual = row['fecha']
        precio_spy_actual = row['close']
        
        # Calcular días hasta vencimiento
        dias_restantes = (vencimiento_dt - fecha_actual).days
        T_restante = max(dias_restantes / 365.0, 1/365.0)  # Mínimo 1 día para evitar división por cero
        
        # Calcular precio actual del call y put usando Black-Scholes propio
        # REUTILIZAMOS bs_price del notebook anterior
        if dias_restantes > 0:
            precio_call_actual = bs_price(precio_spy_actual, strike, T_restante, r, q, sigma, 'C')
            precio_put_actual = bs_price(precio_spy_actual, strike, T_restante, r, q, sigma, 'P')
            valor_straddle_actual = precio_call_actual + precio_put_actual
            
            # Calcular griegas actuales del straddle usando funciones propias
            # REUTILIZAMOS bs_greeks_manual del notebook anterior
            griegas_call = bs_greeks_manual(precio_spy_actual, strike, T_restante, r, q, sigma, 'C')
            griegas_put = bs_greeks_manual(precio_spy_actual, strike, T_restante, r, q, sigma, 'P')
            
            delta_call_actual = griegas_call['delta']
            delta_put_actual = griegas_put['delta']
            delta_straddle = delta_call_actual + delta_put_actual
            
            gamma_straddle = griegas_call['gamma'] + griegas_put['gamma']
            vega_straddle = griegas_call['vega'] + griegas_put['vega']
            theta_straddle = griegas_call['theta'] + griegas_put['theta']
        else:
            # En vencimiento, usar valor intrínseco
            precio_call_actual = max(precio_spy_actual - strike, 0)
            precio_put_actual = max(strike - precio_spy_actual, 0)
            valor_straddle_actual = precio_call_actual + precio_put_actual
            delta_straddle = 0.0  # En vencimiento, delta es 0 o 1/-1 según ITM/OTM
            gamma_straddle = 0.0
            vega_straddle = 0.0
            theta_straddle = 0.0
        
        # Determinar ajuste necesario en la posición de SPY
        # Para neutralizar delta necesitamos: posición_SPY = -Delta_straddle × 100 (por contrato)
        # Si Delta_straddle = +0.20 → necesitamos -20 acciones (posición corta)
        # Si Delta_straddle = -0.15 → necesitamos +15 acciones (posición larga)
        acciones_necesarias = -delta_straddle * 100  # Multiplicador de opciones = 100
        
        # Calcular acciones a comprar/vender
        acciones_a_operar = acciones_necesarias - posicion_spy
        
        # Ejecutar rebalanceo (siempre rebalanceamos en frecuencia diaria)
        if frecuencia == 'diaria' and abs(acciones_a_operar) > 0.01:  # Tolerancia mínima
            # Actualizar posición
            posicion_spy = acciones_necesarias
            
            # Calcular cash flow (negativo si compramos, positivo si vendemos)
            cash_flow = -acciones_a_operar * precio_spy_actual
        else:
            acciones_a_operar = 0
            cash_flow = 0
        
        # Calcular delta neto después del rebalanceo (debería estar cerca de 0)
        delta_neto = delta_straddle + (posicion_spy / 100.0)  # Normalizar por multiplicador
        
        # Calcular P&L diario
        if idx > 0:
            pnl_straddle = valor_straddle_actual - valor_straddle_anterior
            pnl_hedge = (precio_spy_actual - precio_spy_anterior) * posicion_spy_anterior
            pnl_total = pnl_straddle + pnl_hedge
        else:
            pnl_straddle = 0.0
            pnl_hedge = 0.0
            pnl_total = 0.0
        
        # Registrar operación
        historial.append({
            'fecha': fecha_actual,
            'precio_spy': precio_spy_actual,
            'delta_straddle': delta_straddle,
            'acciones_operadas': acciones_a_operar,
            'posicion_spy_total': posicion_spy,
            'delta_neto': delta_neto,
            'cash_flow': cash_flow,
            'valor_straddle': valor_straddle_actual,
            'gamma': gamma_straddle,
            'vega': vega_straddle,
            'theta': theta_straddle,
            'pnl_straddle': pnl_straddle,
            'pnl_hedge': pnl_hedge,
            'pnl_total': pnl_total
        })
        
        # Actualizar valores para siguiente iteración
        valor_straddle_anterior = valor_straddle_actual
        precio_spy_anterior = precio_spy_actual
        posicion_spy_anterior = posicion_spy
    
    # Crear DataFrame con historial
    df_historial = pd.DataFrame(historial)
    
    # Calcular métricas resumen
    rebalanceos_efectivos = df_historial[df_historial['acciones_operadas'] != 0]
    
    metricas = {
        'numero_rebalanceos': len(rebalanceos_efectivos),
        'total_acciones_compradas': rebalanceos_efectivos[rebalanceos_efectivos['acciones_operadas'] > 0]['acciones_operadas'].sum() if len(rebalanceos_efectivos) > 0 else 0,
        'total_acciones_vendidas': abs(rebalanceos_efectivos[rebalanceos_efectivos['acciones_operadas'] < 0]['acciones_operadas'].sum()) if len(rebalanceos_efectivos) > 0 else 0,
        'cash_flow_total': df_historial['cash_flow'].sum(),
        'delta_medio': df_historial['delta_neto'].mean(),
        'delta_maximo': df_historial['delta_neto'].abs().max(),
        'delta_std': df_historial['delta_neto'].std(),
        'pnl_total_acumulado': df_historial['pnl_total'].sum(),
        'pnl_straddle_total': df_historial['pnl_straddle'].sum(),
        'pnl_hedge_total': df_historial['pnl_hedge'].sum()
    }
    
    return {
        'historial_rebalanceos': df_historial,
        'metricas_resumen': metricas
    }


# Implementación: Objetivo 2.2 (mantener clase existente para compatibilidad)

class DeltaHedgedStraddleStrategy(LongStraddleStrategy):
    """
    Estrategia Long Straddle con delta-hedging diario.
    
    EXTENDE LongStraddleStrategy añadiendo cobertura delta.
    REUTILIZA bs_greeks_manual() del código original.
    """
    
    def __init__(self, entry_frequency_days=7, exit_at_expiry=True, 
                 stop_loss_pct=0.50, take_profit_pct=1.0, 
                 hedge_tolerance=0.05, transaction_cost=0.0001):
        super().__init__(entry_frequency_days, exit_at_expiry, stop_loss_pct, take_profit_pct)
        self.hedge_tolerance = hedge_tolerance
        self.transaction_cost = transaction_cost
        
    def backtest_hedged(self, S_prices, dates, r, q, sigma, strike_spacing=5.0):
        """
        Backtest con delta-hedging diario.
        
        Args (usando nomenclatura original):
            S_prices: Precios históricos (variable S)
            dates: Fechas
            r, q, sigma: Parámetros del código original
        """
        self.trades = []
        current_date_idx = 0
        
        while current_date_idx < len(dates) - 30:
            entry_date_idx = current_date_idx
            S_entry = S_prices[entry_date_idx]
            entry_date = dates[entry_date_idx]
            
            K_strike = round(S_entry / strike_spacing) * strike_spacing
            T = 30 / 365.0
            
            # REUTILIZAMOS bs_price del código original
            call_price = bs_price(S_entry, K_strike, T, r, q, sigma, 'C')
            put_price = bs_price(S_entry, K_strike, T, r, q, sigma, 'P')
            total_cost = call_price + put_price
            
            # Inicializar hedge
            hedge_shares = 0
            total_hedge_costs = 0
            rebalance_count = 0
            
            exit_date_idx = min(entry_date_idx + 30, len(dates) - 1)
            exit_reason = 'expiry'
            exit_pnl = None
            
            for day_idx in range(entry_date_idx + 1, exit_date_idx + 1):
                S_current = S_prices[day_idx]
                days_left = 30 - (day_idx - entry_date_idx)
                T_left = days_left / 365.0
                
                if T_left > 0:
                    # REUTILIZAMOS funciones del código original
                    current_call = bs_price(S_current, K_strike, T_left, r, q, sigma, 'C')
                    current_put = bs_price(S_current, K_strike, T_left, r, q, sigma, 'P')
                    current_value = current_call + current_put
                    
                    # Calcular delta total del straddle - REUTILIZAMOS bs_greeks_manual
                    call_greeks = bs_greeks_manual(S_current, K_strike, T_left, r, q, sigma, 'C')
                    put_greeks = bs_greeks_manual(S_current, K_strike, T_left, r, q, sigma, 'P')
                    total_delta = call_greeks['delta'] + put_greeks['delta']
                    
                    # Calcular hedge necesario
                    target_hedge = -total_delta  # Negativo para cubrir
                    hedge_diff = target_hedge - hedge_shares
                    
                    # Rebalancear si es necesario
                    if abs(hedge_diff) > self.hedge_tolerance:
                        cost = abs(hedge_diff) * S_current * self.transaction_cost
                        hedge_shares = target_hedge
                        total_hedge_costs += cost
                        rebalance_count += 1
                else:
                    current_call = max(S_current - K_strike, 0)
                    current_put = max(K_strike - S_current, 0)
                    current_value = current_call + current_put
                
                pnl = current_value - total_cost
                hedge_pnl = hedge_shares * (S_current - S_entry)
                total_pnl = pnl + hedge_pnl - total_hedge_costs
                pnl_pct = total_pnl / total_cost
                
                if pnl_pct <= -self.stop_loss:
                    exit_date_idx = day_idx
                    exit_reason = 'stop_loss'
                    exit_pnl = total_pnl
                    break
                elif pnl_pct >= self.take_profit:
                    exit_date_idx = day_idx
                    exit_reason = 'take_profit'
                    exit_pnl = total_pnl
                    break
            
            if exit_pnl is None:
                S_final = S_prices[exit_date_idx]
                final_call = max(S_final - K_strike, 0)
                final_put = max(K_strike - S_final, 0)
                final_value = final_call + final_put
                hedge_pnl_final = hedge_shares * (S_final - S_entry)
                exit_pnl = (final_value - total_cost) + hedge_pnl_final - total_hedge_costs
            
            self.trades.append({
                'entry_date': entry_date,
                'exit_date': dates[exit_date_idx],
                'S_entry': S_entry,
                'S_exit': S_prices[exit_date_idx],
                'K': K_strike,
                'cost': total_cost,
                'pnl': exit_pnl,
                'pnl_pct': exit_pnl / total_cost,
                'exit_reason': exit_reason,
                'days_held': exit_date_idx - entry_date_idx,
                'rebalances': rebalance_count,
                'hedge_costs': total_hedge_costs
            })
            
            current_date_idx = entry_date_idx + self.entry_frequency
        
        return pd.DataFrame(self.trades)


# Comparación: Straddle normal vs Delta-hedged
print(f"\n{'='*60}")
print(f"COMPARACIÓN: STRADDLE NORMAL vs DELTA-HEDGED")
print(f"{'='*60}\n")

# Usar mismos datos que en objetivo 2.1
strategy_normal = LongStraddleStrategy(entry_frequency_days=7, exit_at_expiry=True)
strategy_hedged = DeltaHedgedStraddleStrategy(entry_frequency_days=7, exit_at_expiry=True)

trades_normal = strategy_normal.backtest(S_sim, dates_sim, r_base, q_base, sigma_base)
trades_hedged = strategy_hedged.backtest_hedged(S_sim, dates_sim, r_base, q_base, sigma_base)

metrics_normal = strategy_normal.calculate_metrics(trades_normal)
metrics_hedged = strategy_hedged.calculate_metrics(trades_hedged)

print(f"RESULTADOS COMPARATIVOS:")
print(f"{'='*60}")
print(f"STRADDLE NORMAL:")
for key, value in metrics_normal.items():
    if isinstance(value, float):
        if 'rate' in key or 'ratio' in key or 'factor' in key:
            print(f"  {key}: {value:.4f}")
        elif 'return' in key or 'drawdown' in key:
            print(f"  {key}: ${value:.2f}")
        else:
            print(f"  {key}: ${value:.2f}")
    else:
        print(f"  {key}: {value}")

print(f"\nSTRADDLE DELTA-HEDGED:")
for key, value in metrics_hedged.items():
    if isinstance(value, float):
        if 'rate' in key or 'ratio' in key or 'factor' in key:
            print(f"  {key}: {value:.4f}")
        elif 'return' in key or 'drawdown' in key:
            print(f"  {key}: ${value:.2f}")
        else:
            print(f"  {key}: ${value:.2f}")
    else:
        print(f"  {key}: {value}")

print(f"\nCostos adicionales de hedging:")
print(f"  Rebalances promedio: {trades_hedged['rebalances'].mean():.1f} por trade")
print(f"  Costos promedio: ${trades_hedged['hedge_costs'].mean():.2f} por trade")
print(f"{'='*60}\n")

# Visualización comparativa
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

ax = axes[0]
equity_normal = trades_normal['pnl'].cumsum()
equity_hedged = trades_hedged['pnl'].cumsum()
ax.plot(range(len(equity_normal)), equity_normal, 'b-', linewidth=2, label='Straddle Normal')
ax.plot(range(len(equity_hedged)), equity_hedged, 'r-', linewidth=2, label='Straddle Delta-Hedged')
ax.set_xlabel('Trade #', fontsize=11)
ax.set_ylabel('P&L Acumulado ($)', fontsize=11)
ax.set_title('Equity Curves Comparativas', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.hist(trades_normal['pnl'], bins=20, alpha=0.5, label='Normal', color='blue')
ax.hist(trades_hedged['pnl'], bins=20, alpha=0.5, label='Delta-Hedged', color='red')
ax.set_xlabel('P&L por Trade ($)', fontsize=11)
ax.set_ylabel('Frecuencia', fontsize=11)
ax.set_title('Distribución de Returns Comparativa', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/pnl_analysis/straddle_hedged_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Gráfico comparativo guardado")


### 2.2. Ejemplo de Uso

Esta celda demuestra cómo usar la función `aplicar_delta_hedge` con un straddle construido y datos históricos de SPY. El ejemplo muestra:
- Construcción de un straddle usando la función del Prompt 1
- Obtención de precios históricos de SPY desde la fecha del straddle hasta vencimiento
- Aplicación de delta-hedging con la función implementada
- Visualización del historial de rebalanceos y métricas resumen


In [ ]:
# Ejemplo de uso de aplicar_delta_hedge

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

print(f"\n{'='*60}")
print(f"EJEMPLO DE USO: DELTA-HEDGING DE LONG STRADDLE")
print(f"{'='*60}\n")

# 1. Construir un straddle (usando función del Prompt 1)
print("Paso 1: Construyendo Long Straddle...")
straddle_info = construir_long_straddle(ticker='SPY', dias_vencimiento=30)
print(f"  Straddle construido el {straddle_info['fecha']}")
print(f"  Strike: ${straddle_info['strike']:.2f}")
print(f"  Inversión inicial: ${straddle_info['inversion_inicial']:.2f}")
print(f"  Vencimiento: {straddle_info['vencimiento']}\n")

# 2. Simular precios históricos de SPY (en un entorno real, obtendrías estos del broker)
print("Paso 2: Obteniendo precios históricos de SPY...")
fecha_inicio = straddle_info['fecha']
fecha_fin = straddle_info['vencimiento']

# Simular trayectoria de precios con movimiento aleatorio
# En un entorno real, usarías: ib.reqHistoricalData() o yfinance
np.random.seed(42)
precio_inicial = straddle_info['precio_subyacente']
dias_totales = (fecha_fin - fecha_inicio).days
fechas = [fecha_inicio + timedelta(days=i) for i in range(dias_totales + 1)]

# Simular precios con movimiento browniano geométrico
dt = 1/252  # Un día de trading
mu = 0.08  # Drift anual
sigma_sim = 0.20  # Volatilidad anual
precios = [precio_inicial]
for i in range(dias_totales):
    cambio = np.random.normal(mu * dt, sigma_sim * np.sqrt(dt))
    nuevo_precio = precios[-1] * np.exp(cambio)
    precios.append(nuevo_precio)

precios_historicos_spy = pd.DataFrame({
    'fecha': fechas,
    'close': precios,
    'open': precios,
    'high': [p * 1.01 for p in precios],
    'low': [p * 0.99 for p in precios],
    'volume': [1000000] * len(precios)
})

print(f"  Obtenidos {len(precios_historicos_spy)} días de datos")
print(f"  Precio inicial: ${precios_historicos_spy['close'].iloc[0]:.2f}")
print(f"  Precio final: ${precios_historicos_spy['close'].iloc[-1]:.2f}\n")

# 3. Aplicar delta-hedging
print("Paso 3: Aplicando delta-hedging diario...")
resultado_hedge = aplicar_delta_hedge(straddle_info, precios_historicos_spy, frecuencia='diaria')

historial = resultado_hedge['historial_rebalanceos']
metricas = resultado_hedge['metricas_resumen']

print(f"  Delta-hedging completado")
print(f"  Total de días analizados: {len(historial)}\n")

# 4. Mostrar historial de rebalanceos (primeras y últimas 10 filas)
print(f"{'='*60}")
print(f"HISTORIAL DE REBALANCEOS (Primeras 10 filas)")
print(f"{'='*60}")
columnas_display = ['fecha', 'precio_spy', 'delta_straddle', 'acciones_operadas', 
                    'posicion_spy_total', 'delta_neto', 'valor_straddle']
print(historial[columnas_display].head(10).to_string(index=False))

print(f"\n{'='*60}")
print(f"HISTORIAL DE REBALANCEOS (Últimas 10 filas)")
print(f"{'='*60}")
print(historial[columnas_display].tail(10).to_string(index=False))

# 5. Mostrar métricas resumen
print(f"\n{'='*60}")
print(f"MÉTRICAS RESUMEN")
print(f"{'='*60}")
print(f"Número total de rebalanceos: {metricas['numero_rebalanceos']}")
print(f"Total acciones compradas: {metricas['total_acciones_compradas']:.2f}")
print(f"Total acciones vendidas: {metricas['total_acciones_vendidas']:.2f}")
print(f"Cash flow total del hedging: ${metricas['cash_flow_total']:.2f}")
print(f"Delta medio después de rebalanceos: {metricas['delta_medio']:.6f}")
print(f"Delta máximo registrado: {metricas['delta_maximo']:.6f}")
print(f"Desviación estándar del delta neto: {metricas['delta_std']:.6f}")
print(f"P&L total acumulado: ${metricas['pnl_total_acumulado']:.2f}")
print(f"P&L del straddle: ${metricas['pnl_straddle_total']:.2f}")
print(f"P&L del hedge: ${metricas['pnl_hedge_total']:.2f}")
print(f"{'='*60}\n")


### 2.3. Visualizaciones del Delta-Hedging

Esta sección crea visualizaciones detalladas que muestran:
- Evolución de la posición de hedge y delta del straddle
- Delta neto a lo largo del tiempo (objetivo: mantener cerca de 0)
- Evolución de las griegas que NO se cubren (Gamma, Vega, Theta)
- Tabla resumen de cash flows del hedging


In [ ]:
# Visualizaciones del delta-hedging

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import os

# Crear directorio para imágenes si no existe
os.makedirs('images/pnl_analysis', exist_ok=True)

# Usar el resultado del ejemplo anterior, si no existe ejecutar ejemplo básico
try:
    # Verificar si resultado_hedge existe en el namespace global
    resultado_hedge
except NameError:
    # Si no existe, ejecutar ejemplo básico
    straddle_info = construir_long_straddle(ticker='SPY', dias_vencimiento=30)
    fecha_inicio = straddle_info['fecha']
    fecha_fin = straddle_info['vencimiento']
    precio_inicial = straddle_info['precio_subyacente']
    dias_totales = (fecha_fin - fecha_inicio).days
    fechas = [fecha_inicio + timedelta(days=i) for i in range(dias_totales + 1)]
    np.random.seed(42)
    dt = 1/252
    mu = 0.08
    sigma_sim = 0.20
    precios = [precio_inicial]
    for i in range(dias_totales):
        cambio = np.random.normal(mu * dt, sigma_sim * np.sqrt(dt))
        nuevo_precio = precios[-1] * np.exp(cambio)
        precios.append(nuevo_precio)
    precios_historicos_spy = pd.DataFrame({
        'fecha': fechas,
        'close': precios
    })
    resultado_hedge = aplicar_delta_hedge(straddle_info, precios_historicos_spy, frecuencia='diaria')

historial = resultado_hedge['historial_rebalanceos']

# Gráfico 1: Posición de hedge y delta del straddle
fig, ax1 = plt.subplots(figsize=(14, 6))

ax1.set_xlabel('Fecha', fontsize=12, fontweight='bold')
ax1.set_ylabel('Número de Acciones SPY', fontsize=12, fontweight='bold', color='blue')
ax1.plot(historial['fecha'], historial['posicion_spy_total'], 'b-', linewidth=2, label='Posición SPY (hedge)')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()
ax2.set_ylabel('Delta del Straddle', fontsize=12, fontweight='bold', color='red')
ax2.plot(historial['fecha'], historial['delta_straddle'], 'r--', linewidth=2, alpha=0.7, label='Delta Straddle')
ax2.tick_params(axis='y', labelcolor='red')

ax1.set_title('Evolución de la Posición de Hedge y Delta del Straddle', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.savefig('images/pnl_analysis/delta_hedging_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Gráfico 1 guardado: Evolución de posición de hedge")

# Gráfico 2: Delta neto
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(historial['fecha'], historial['delta_neto'], 'g-', linewidth=2, label='Delta Neto (Straddle + Hedge)')
ax.axhline(0, color='black', linestyle='-', linewidth=1, alpha=0.5, label='Objetivo (Delta = 0)')
ax.axhline(0.1, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='Banda ±0.1')
ax.axhline(-0.1, color='orange', linestyle='--', linewidth=1, alpha=0.5)
ax.fill_between(historial['fecha'], -0.1, 0.1, alpha=0.1, color='green', label='Zona objetivo')

ax.set_xlabel('Fecha', fontsize=12, fontweight='bold')
ax.set_ylabel('Delta Neto', fontsize=12, fontweight='bold')
ax.set_title('Delta Neto: Efectividad del Delta-Hedging', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/pnl_analysis/delta_neto_evolution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Gráfico 2 guardado: Delta neto")

# Gráfico 3: Evolución de griegas que NO se cubren
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(historial['fecha'], historial['gamma'], 'b-', linewidth=2, label='Gamma')
ax.plot(historial['fecha'], historial['vega'], 'g-', linewidth=2, label='Vega')
ax.plot(historial['fecha'], historial['theta'], 'r-', linewidth=2, label='Theta')

ax.set_xlabel('Fecha', fontsize=12, fontweight='bold')
ax.set_ylabel('Valor de la Griega', fontsize=12, fontweight='bold')
ax.set_title('Exposiciones Remanentes en Estrategia Delta-Hedged', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/pnl_analysis/greeks_evolution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Gráfico 3 guardado: Evolución de griegas")

# Tabla resumen de cash flows
print(f"\n{'='*60}")
print(f"TABLA RESUMEN DE CASH FLOWS DEL HEDGING")
print(f"{'='*60}")

rebalanceos_con_cashflow = historial[historial['acciones_operadas'] != 0].copy()
rebalanceos_con_cashflow['accion'] = rebalanceos_con_cashflow['acciones_operadas'].apply(
    lambda x: 'Compra' if x > 0 else 'Venta'
)
rebalanceos_con_cashflow['cash_flow_acumulado'] = rebalanceos_con_cashflow['cash_flow'].cumsum()

tabla_cashflow = pd.DataFrame({
    'Fecha': rebalanceos_con_cashflow['fecha'].dt.strftime('%Y-%m-%d'),
    'Acción': rebalanceos_con_cashflow['accion'],
    'Acciones': rebalanceos_con_cashflow['acciones_operadas'].apply(lambda x: f"{x:.2f}"),
    'Precio SPY': rebalanceos_con_cashflow['precio_spy'].apply(lambda x: f"${x:.2f}"),
    'Cash Flow': rebalanceos_con_cashflow['cash_flow'].apply(lambda x: f"${x:.2f}"),
    'Cash Flow Acumulado': rebalanceos_con_cashflow['cash_flow_acumulado'].apply(lambda x: f"${x:.2f}")
})

print(tabla_cashflow.to_string(index=False))
print(f"{'='*60}\n")


### 2.4. Conceptos Clave del Delta-Hedging

#### ¿Por qué delta-hedging NO elimina todo el riesgo?

Delta es solo la sensibilidad de primer orden (derivada primera). Gamma (segunda derivada) causa que delta cambie, por lo que siempre tenemos error de cobertura. Entre rebalanceos, acumulamos exposición a movimientos. Además, Vega nos expone a cambios en volatilidad implícita, y Theta nos hace perder dinero cada día que pasa.

#### ¿Qué riesgos quedan después de neutralizar delta?

1. **Riesgo de Gamma**: Beneficio de movimientos grandes, pérdida si el mercado no se mueve
2. **Riesgo de Vega**: Beneficio si IV sube, pérdida si IV baja
3. **Riesgo de Theta**: Pérdida constante por el paso del tiempo
4. **Riesgo de modelo**: Nuestras griegas de Black-Scholes pueden ser incorrectas
5. **Riesgo de ejecución**: No siempre podemos rebalancear al precio exacto
6. **Riesgo de gap**: Movimientos durante el cierre del mercado

#### Gamma scalping implícito

Cada rebalanceo es una operación de gamma scalping:
- Vendemos acciones cuando el mercado sube (realizamos beneficio del call)
- Compramos acciones cuando el mercado baja (realizamos beneficio del put)
- El beneficio acumulado de gamma scalping debe superar el theta decay para ganar dinero
- Fórmula aproximada: P&L ≈ 0.5 × Gamma × (ΔS)² - Theta × Δt

En mercados volátiles con muchos movimientos, el gamma scalping puede generar beneficios significativos que compensan el theta decay. En mercados tranquilos, el theta decay domina y la estrategia pierde dinero.


## Objetivo 3: Análisis del P&L Histórico de Ambas Versiones

### Descripción

Este objetivo realiza un análisis exhaustivo del P&L histórico comparando la estrategia Long Straddle normal vs la versión delta-hedged. El análisis incluye métricas de rendimiento, análisis de riesgo, y visualizaciones comparativas que permiten entender las diferencias entre ambas aproximaciones.

Analizamos métricas clave como total return, win rate, average win/loss, Sharpe ratio anualizado, profit factor (gross profit / gross loss), maximum drawdown, volatilidad del P&L, y costos de transacción. Realizamos visualizaciones comparativas incluyendo equity curves superpuestas, distribución de returns, análisis de drawdown, y desglose de ganancias/pérdidas por período.

**Funciones reutilizadas**: Todas las funciones de las estrategias anteriores (`LongStraddleStrategy` y `DeltaHedgedStraddleStrategy`) y las funciones de cálculo de griegas `bs_greeks_manual()`.

**Métricas analizadas**: Total return, win rate, average win/loss, Sharpe ratio, profit factor, maximum drawdown, volatilidad del P&L, número de trades, días promedio en posición, y costos de transacción acumulados.

**Resultados esperados**: Tablas comparativas de métricas, gráficos de equity curves superpuestas, análisis de distribución de returns, comparación de drawdowns, y conclusiones sobre qué versión funciona mejor en diferentes condiciones de mercado.

### Descripción

Hasta ahora, hemos neutralizado el delta usando el subyacente (acciones de SPY). Sin embargo, también es posible neutralizar el delta de una opción usando **otra opción**. Este enfoque tiene implicaciones importantes en todas las griegas, no solo en el delta. Usar otra opción en lugar del subyacente ofrece flexibilidad (puedes elegir strikes y tipos de opción que modifiquen otras griegas), eficiencia de capital (a veces más eficiente que comprar/vender el subyacente), control de Gamma y Vega (puedes ajustar la exposición a estas griegas simultáneamente), y es la base para estrategias avanzadas como spreads, butterflies, condors.

Analizamos las consecuencias de neutralizar delta con otra opción: cálculo del ratio de cobertura (cuántas opciones de cobertura necesitamos), impacto en otras griegas (Gamma: ¿aumenta o disminuye?, Theta: ¿ganamos o perdemos tiempo?, Vega: ¿aumenta o disminuye la exposición a volatilidad?), y comparación con hedging con subyacente (ventajas y desventajas).

Para el cálculo del ratio de cobertura, si tenemos 1 opción principal con delta δ₁ y queremos usar otra opción con delta δ₂, necesitamos N opciones de cobertura tal que: δ₁ + N × δ₂ = 0, por lo tanto N = -δ₁ / δ₂. Analizamos griegas combinadas: delta combinado δ₁ + N × δ₂ (debe ser ≈ 0), gamma combinado γ₁ + N × γ₂, theta combinado θ₁ + N × θ₂, y vega combinado ν₁ + N × ν₂. Visualizamos el payoff diagram de la combinación y comparamos griegas antes y después.

**Funciones reutilizadas**: `bs_greeks_manual(S, K, T, r, q, sigma, right)` para calcular todas las griegas de ambas opciones.

**Ejemplo típico**: Opción principal Long Call ATM (delta ≈ 0.5), opción de cobertura Short Call OTM (delta ≈ 0.3), ratio -0.5 / 0.3 = -1.67 (necesitamos vender 1.67 Calls OTM).

**Implicaciones**: Gamma (si la opción de cobertura tiene gamma positivo, el gamma combinado puede aumentar), Theta (depende de si la opción de cobertura es long o short), Vega (similar a theta, depende de la posición), y Payoff (el payoff combinado es diferente al de una sola opción).

**Resultados esperados**: Cálculo del ratio de cobertura necesario, análisis de cómo cambian todas las griegas, payoff diagram de la combinación, comparación visual de griegas antes/después, y recomendaciones sobre cuándo usar este método vs hedging con subyacente.



In [ ]:
# Implementación: Función construir_long_straddle

def construir_long_straddle(ticker='SPY', dias_vencimiento=30, fecha_analisis=None, ib=None):
    """
    Construye un Long Straddle sobre el ticker especificado.
    
    Esta función intenta obtener datos del broker si están disponibles,
    o usa datos del notebook EJERCICIO_MIAX_2025 si ya fueron obtenidos.
    Si no hay datos disponibles, usa valores simulados.
    
    Args:
        ticker: Símbolo del activo (por defecto 'SPY')
        dias_vencimiento: Días hasta el vencimiento deseado (aproximado)
        fecha_analisis: Fecha en la que se construye el straddle (por defecto hoy)
        ib: Objeto IB de ib_insync si está conectado al broker (opcional)
    
    Returns:
        Diccionario con toda la información del straddle construido
    """
    from datetime import datetime, timedelta
    
    # Si no se especifica fecha, usar hoy
    if fecha_analisis is None:
        fecha_analisis = datetime.now()
    
    # Intentar obtener precio actual del subyacente
    # Prioridad: 1) Broker conectado, 2) Datos del notebook anterior, 3) Simulación
    precio_subyacente = None
    
    # Opción 1: Intentar obtener del broker si está conectado
    if ib is not None:
        try:
            from ib_insync import Stock
            spy = Stock(ticker, 'SMART', 'USD')
            ib.qualifyContracts(spy)
            ticker_spy = ib.reqMktData(spy, '', False, False)
            ib.sleep(1)  # Esperar a que llegue el precio
            if ticker_spy.marketPrice():
                precio_subyacente = ticker_spy.marketPrice()
                print(f"[INFO] Precio obtenido del broker: ${precio_subyacente:.2f}")
        except Exception as e:
            print(f"[INFO] No se pudo obtener precio del broker: {e}")
    
    # Opción 2: Intentar usar datos del notebook anterior si existen
    if precio_subyacente is None:
        try:
            # Buscar si hay una variable S (precio) del notebook anterior
            if 'S' in globals() or 'S' in dir():
                precio_subyacente = globals().get('S', locals().get('S', None))
                if precio_subyacente:
                    print(f"[INFO] Precio obtenido de datos previos: ${precio_subyacente:.2f}")
        except:
            pass
    
    # Opción 3: Usar valor por defecto si no hay datos disponibles
    if precio_subyacente is None:
        precio_subyacente = 450.0
        print(f"[INFO] Usando precio simulado: ${precio_subyacente:.2f}")
    
    # Calcular fecha de vencimiento aproximada
    # En un entorno real, buscaríamos el vencimiento más cercano a dias_vencimiento
    vencimiento = fecha_analisis + timedelta(days=dias_vencimiento)
    dias_hasta_vencimiento = (vencimiento - fecha_analisis).days
    T = dias_hasta_vencimiento / 365.0
    
    # Identificar strike ATM: el strike más cercano al precio actual
    # En un entorno real, esto se haría consultando la cadena de opciones del broker
    strike_spacing = 5.0  # Espaciado típico de $5 para SPY
    strike = round(precio_subyacente / strike_spacing) * strike_spacing
    
    # Obtener parámetros de mercado
    # Intentar usar datos del notebook anterior, sino usar valores por defecto
    try:
        r = globals().get('r', locals().get('r', 0.05))
        q = globals().get('q', locals().get('q', 0.0))
        sigma = globals().get('sigma', locals().get('sigma', 0.15))
    except:
        r = 0.05
        q = 0.0
        sigma = 0.15
    
    # Calcular precios de Call y Put usando bs_price del notebook anterior
    # REUTILIZAMOS la función bs_price del código original
    precio_call = bs_price(precio_subyacente, strike, T, r, q, sigma, 'C')
    precio_put = bs_price(precio_subyacente, strike, T, r, q, sigma, 'P')
    
    # Intentar obtener precios reales del broker si está conectado
    precio_call_bid = None
    precio_call_ask = None
    precio_put_bid = None
    precio_put_ask = None
    
    if ib is not None:
        try:
            from ib_insync import Option
            # Crear contratos de opciones
            opt_call = Option(ticker, vencimiento.strftime('%Y%m%d'), strike, 'C', 'SMART')
            opt_put = Option(ticker, vencimiento.strftime('%Y%m%d'), strike, 'P', 'SMART')
            ib.qualifyContracts(opt_call, opt_put)
            
            # Obtener precios de mercado
            ticker_call = ib.reqMktData(opt_call, '', False, False)
            ticker_put = ib.reqMktData(opt_put, '', False, False)
            ib.sleep(1)
            
            if ticker_call.bid and ticker_call.ask:
                precio_call_bid = ticker_call.bid
                precio_call_ask = ticker_call.ask
                print(f"[INFO] Precios Call obtenidos del broker: bid=${precio_call_bid:.2f}, ask=${precio_call_ask:.2f}")
            
            if ticker_put.bid and ticker_put.ask:
                precio_put_bid = ticker_put.bid
                precio_put_ask = ticker_put.ask
                print(f"[INFO] Precios Put obtenidos del broker: bid=${precio_put_bid:.2f}, ask=${precio_put_ask:.2f}")
        except Exception as e:
            print(f"[INFO] No se pudieron obtener precios del broker: {e}")
    
    # Si no hay precios del broker, usar precios teóricos con simulación de spread
    if precio_call_bid is None:
        precio_call_bid = precio_call * 0.99  # Simulación: bid ligeramente menor
        precio_call_ask = precio_call * 1.01  # Simulación: ask ligeramente mayor
    
    if precio_put_bid is None:
        precio_put_bid = precio_put * 0.99
        precio_put_ask = precio_put * 1.01
    
    precio_call_mid = (precio_call_bid + precio_call_ask) / 2
    precio_put_mid = (precio_put_bid + precio_put_ask) / 2
    
    # Calcular griegas individuales usando bs_greeks_manual del notebook anterior
    # REUTILIZAMOS la función bs_greeks_manual del código original
    griegas_call = bs_greeks_manual(precio_subyacente, strike, T, r, q, sigma, 'C')
    griegas_put = bs_greeks_manual(precio_subyacente, strike, T, r, q, sigma, 'P')
    
    # Calcular griegas totales del straddle
    # Delta total: Call + Put (debería estar cerca de 0 si es verdadero ATM)
    delta_total = griegas_call['delta'] + griegas_put['delta']
    
    # Gamma total: suma de gammas (siempre positivo)
    gamma_total = griegas_call['gamma'] + griegas_put['gamma']
    
    # Theta total: suma de thetas (siempre negativo, costo temporal)
    theta_total = griegas_call['theta'] + griegas_put['theta']
    
    # Vega total: suma de vegas (siempre positivo, beneficio de aumento de IV)
    vega_total = griegas_call['vega'] + griegas_put['vega']
    
    # Rho total: suma de rhos (normalmente cerca de 0, efectos opuestos)
    rho_total = griegas_call['rho'] + griegas_put['rho']
    
    # Calcular inversión inicial (prima total pagada)
    inversion_inicial = precio_call_mid + precio_put_mid
    
    # Calcular puntos de equilibrio
    breakeven_superior = strike + inversion_inicial
    breakeven_inferior = strike - inversion_inicial
    rango_beneficio = breakeven_superior - breakeven_inferior
    
    # Información de contratos
    call_contract = {
        'symbol': f'{ticker}',
        'strike': strike,
        'expiry': vencimiento,
        'right': 'C',
        'bid': precio_call_bid,
        'ask': precio_call_ask,
        'mid': precio_call_mid
    }
    
    put_contract = {
        'symbol': f'{ticker}',
        'strike': strike,
        'expiry': vencimiento,
        'right': 'P',
        'bid': precio_put_bid,
        'ask': precio_put_ask,
        'mid': precio_put_mid
    }
    
    # Retornar diccionario con toda la información
    return {
        'fecha': fecha_analisis,
        'precio_subyacente': precio_subyacente,
        'vencimiento': vencimiento,
        'dias_hasta_vencimiento': dias_hasta_vencimiento,
        'strike': strike,
        'call_contract': call_contract,
        'put_contract': put_contract,
        'precio_call': precio_call_mid,
        'precio_put': precio_put_mid,
        'inversion_inicial': inversion_inicial,
        'griegas_call': griegas_call,
        'griegas_put': griegas_put,
        'griegas_totales': {
            'delta': delta_total,
            'gamma': gamma_total,
            'theta': theta_total,
            'vega': vega_total,
            'rho': rho_total
        },
        'breakeven_superior': breakeven_superior,
        'breakeven_inferior': breakeven_inferior,
        'rango_beneficio': rango_beneficio
    }

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# OBJETIVO 2.5: NEUTRALIZACIÓN CON OTRA OPCIÓN
# ═══════════════════════════════════════════════════════════════════════

def neutralize_delta_with_option(S, K1, K2, T, r, q, sigma, right1='C', right2='C'):
    """
    Neutraliza delta de una opción usando otra opción.
    
    REUTILIZA bs_greeks_manual() del código original.
    
    Args (nomenclatura original):
        S: Precio spot
        K1: Strike de la opción principal
        K2: Strike de la opción de cobertura
        T: Tiempo hasta vencimiento
        r, q, sigma: Parámetros del código original
        right1: Tipo de opción principal ('C' o 'P')
        right2: Tipo de opción de cobertura ('C' o 'P')
    
    Returns:
        dict: Análisis completo de griegas antes y después
    """
    print(f"\n{'='*60}")
    print(f"NEUTRALIZACIÓN DE DELTA CON OTRA OPCIÓN")
    print(f"{'='*60}\n")
    
    # REUTILIZAMOS función del código original
    greeks1 = bs_greeks_manual(S, K1, T, r, q, sigma, right1)
    greeks2 = bs_greeks_manual(S, K2, T, r, q, sigma, right2)
    
    delta1 = greeks1['delta']
    delta2 = greeks2['delta']
    
    # Calcular ratio necesario para neutralizar
    # Si tenemos 1 opción principal, necesitamos N opciones de cobertura tal que:
    # delta1 + N * delta2 = 0
    # N = -delta1 / delta2
    
    if abs(delta2) < 1e-6:
        print("  Delta de la opción de cobertura es muy pequeña. No se puede neutralizar.")
        return None
    
    hedge_ratio = -delta1 / delta2
    
    print(f"Opción Principal:")
    print(f"  Tipo: {right1}, Strike: ${K1:.2f}")
    print(f"  Delta: {delta1:.4f}")
    print(f"  Gamma: {greeks1['gamma']:.6f}")
    print(f"  Theta: ${greeks1['theta']:.4f}/día")
    print(f"  Vega: ${greeks1['vega']:.4f}")
    
    print(f"\nOpción de Cobertura:")
    print(f"  Tipo: {right2}, Strike: ${K2:.2f}")
    print(f"  Delta: {delta2:.4f}")
    print(f"  Gamma: {greeks2['gamma']:.6f}")
    print(f"  Theta: ${greeks2['theta']:.4f}/día")
    print(f"  Vega: ${greeks2['vega']:.4f}")
    
    print(f"\nRatio de Cobertura:")
    print(f"  Necesitamos {abs(hedge_ratio):.4f} opciones de cobertura por cada opción principal")
    print(f"  Signo: {'Short' if hedge_ratio < 0 else 'Long'}")
    
    # Calcular griegas combinadas
    gamma_combined = greeks1['gamma'] + hedge_ratio * greeks2['gamma']
    theta_combined = greeks1['theta'] + hedge_ratio * greeks2['theta']
    vega_combined = greeks1['vega'] + hedge_ratio * greeks2['vega']
    
    print(f"\n{'='*60}")
    print(f"GRIEGAS COMBINADAS (Después de neutralizar Delta)")
    print(f"{'='*60}")
    print(f"Delta: {delta1 + hedge_ratio * delta2:.6f} (debe ser ~0)")
    print(f"Gamma: {gamma_combined:.6f} ({'Aumenta' if gamma_combined > greeks1['gamma'] else 'Disminuye'})")
    print(f"Theta: ${theta_combined:.4f}/día ({'Ganamos' if theta_combined > 0 else 'Perdemos'} tiempo)")
    print(f"Vega: ${vega_combined:.4f} ({'Net Long' if vega_combined > 0 else 'Net Short'} volatilidad)")
    print(f"{'='*60}\n")
    
    # Análisis de implicaciones
    print(f"ANÁLISIS DE IMPLICACIONES:")
    print(f"{'='*60}")
    
    if gamma_combined > greeks1['gamma']:
        print(f"  Gamma AUMENTA: Mayor exposición a movimientos del mercado")
    else:
        print(f"✓ Gamma DISMINUYE: Menor exposición a movimientos del mercado")
    
    if theta_combined > 0:
        print(f"✓ Theta POSITIVO: Ganamos dinero con el paso del tiempo")
    else:
        print(f"  Theta NEGATIVO: Perdemos dinero con el paso del tiempo")
    
    if abs(vega_combined) > abs(greeks1['vega']):
        print(f"  Vega AUMENTA: Mayor exposición a cambios de volatilidad")
    else:
        print(f"✓ Vega DISMINUYE: Menor exposición a cambios de volatilidad")
    
    print(f"{'='*60}\n")
    
    return {
        'hedge_ratio': hedge_ratio,
        'delta_combined': delta1 + hedge_ratio * delta2,
        'gamma_combined': gamma_combined,
        'theta_combined': theta_combined,
        'vega_combined': vega_combined,
        'greeks_before': greeks1,
        'greeks_hedge': greeks2
    }


# Ejemplo: Long Call ATM neutralizado con Short Call OTM
try:
    S_ex = S if 'S' in dir() else 450.0
    K1_ex = K if 'K' in dir() else 450.0
    T_ex = T if 'T' in dir() else 30/365.0
    r_ex = r if 'r' in dir() else 0.05
    q_ex = q if 'q' in dir() else 0.0
    sigma_ex = sigma if 'sigma' in dir() else 0.15
except:
    S_ex = 450.0
    K1_ex = 450.0
    T_ex = 30/365.0
    r_ex = 0.05
    q_ex = 0.0
    sigma_ex = 0.15

# Call ATM (delta ≈ 0.5)
K1 = S_ex
# Call OTM (delta ≈ 0.3)
K2 = S_ex * 1.05

result = neutralize_delta_with_option(S_ex, K1, K2, T_ex, r_ex, q_ex, sigma_ex, 'C', 'C')

# Visualización
if result:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Payoff combinado
    ax = axes[0]
    strikes_range = np.linspace(S_ex * 0.85, S_ex * 1.15, 100)
    
    # Payoff de Call ATM
    payoff1 = np.maximum(strikes_range - K1, 0)
    # Payoff de Call OTM (short, ratio negativo)
    payoff2 = -abs(result['hedge_ratio']) * np.maximum(strikes_range - K2, 0)
    payoff_combined = payoff1 + payoff2
    
    ax.plot(strikes_range, payoff1, 'b--', linewidth=1.5, label='Call ATM (Long)')
    ax.plot(strikes_range, payoff2, 'r--', linewidth=1.5, label=f'Call OTM (Short x{abs(result["hedge_ratio"]):.2f})')
    ax.plot(strikes_range, payoff_combined, 'g-', linewidth=2, label='Payoff Combinado')
    ax.axhline(0, color='black', linestyle='--', alpha=0.5)
    ax.axvline(K1, color='b', linestyle=':', alpha=0.5, label=f'Strike 1: ${K1:.2f}')
    ax.axvline(K2, color='r', linestyle=':', alpha=0.5, label=f'Strike 2: ${K2:.2f}')
    ax.set_xlabel('Precio del Subyacente al Vencimiento ($)', fontsize=11)
    ax.set_ylabel('Payoff ($)', fontsize=11)
    ax.set_title('Payoff Diagram: Neutralización con Opción', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Comparación de griegas
    ax = axes[1]
    greeks_names = ['Delta', 'Gamma', 'Theta', 'Vega']
    before_values = [
        result['greeks_before']['delta'],
        result['greeks_before']['gamma'],
        result['greeks_before']['theta'],
        result['greeks_before']['vega']
    ]
    after_values = [
        result['delta_combined'],
        result['gamma_combined'],
        result['theta_combined'],
        result['vega_combined']
    ]
    
    x = np.arange(len(greeks_names))
    width = 0.35
    
    ax.bar(x - width/2, before_values, width, label='Antes (Solo Principal)', alpha=0.7)
    ax.bar(x + width/2, after_values, width, label='Después (Combinado)', alpha=0.7)
    ax.set_xlabel('Griega', fontsize=11)
    ax.set_ylabel('Valor', fontsize=11)
    ax.set_title('Comparación de Griegas: Antes vs Después', fontsize=12, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(greeks_names)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('images/pnl_analysis/option_neutralization.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✓ Gráfico de neutralización guardado")

print("\n Objetivo 2.5 completado: Neutralización con otra opción")



## Objetivo 4: Simulación de Envío de Órdenes (Combo vs Patas Separadas)

### Descripción

En el trading real de opciones, existen dos formas principales de ejecutar estrategias multi-leg (como un straddle): **Orden Combo** (se envía como un paquete único, ejecutando todas las patas simultáneamente) y **Orden Legged** (se ejecuta cada pata por separado, con latencia entre ellas). Este objetivo simula ambos métodos y analiza el **legging risk**, que es el riesgo de que el precio cambie entre la ejecución de las diferentes patas de la estrategia.

El análisis es importante porque el **slippage** (diferencia entre el precio esperado y el precio ejecutado), el **legging risk** (riesgo de que el precio cambie entre la ejecución de patas separadas), la **latencia** (tiempo entre la ejecución de diferentes patas puede resultar en precios adversos), y los **costos de ejecución** (los combos suelen tener menor slippage pero pueden tener spreads más amplios) afectan significativamente el rendimiento de la estrategia.

Para la simulación de orden combo, calculamos el precio neto del straddle (Call + Put), aplicamos slippage como fracción del precio, y simulamos latencia mínima (ej: 50ms). Para la simulación de orden legged, ejecutamos la primera pata (Call) al precio actual, simulamos latencia (ej: 2 segundos), durante la cual el precio puede cambiar, ejecutamos la segunda pata (Put) al nuevo precio, y calculamos el legging risk (diferencia entre precio esperado y ejecutado). Realizamos un análisis estadístico comparando distribuciones de slippage vs legging risk, calculando probabilidad de ejecución adversa, y analizando el impacto en el costo total de la estrategia.

**Funciones reutilizadas**: `bs_price(S, K, T, r, q, sigma, right)` para calcular precios teóricos de Call y Put.

**Parámetros de simulación**: Slippage (fracción del precio, ej: 0.001 = 0.1%), latencia (tiempo entre patas en órdenes legged, ej: 2 segundos), volatilidad del precio (simula cambios de precio durante la latencia), y número de simulaciones (para obtener distribuciones estadísticas, ej: 100 escenarios).

**Resultados esperados**: Distribución de diferencias de precio entre métodos, comparación de costos promedio y máximos, análisis de probabilidad de ejecución adversa, y recomendaciones sobre cuándo usar cada método.

### Descripción

Aunque SPY (ETF) y SPX (índice) representan el mismo subyacente económico (S&P 500), las opciones sobre cada uno tienen características muy diferentes que afectan significativamente la estrategia de trading, los costos, y los resultados fiscales. **SPY (SPDR S&P 500 ETF Trust)** es un ETF que replica el índice S&P 500, se negocia como una acción normal, y tiene opciones con ejercicio estilo americano. **SPX (S&P 500 Index)** es el índice mismo (no un instrumento negociable directamente), tiene opciones con ejercicio estilo europeo, y liquidación en efectivo (cash settlement).

Proporcionamos un análisis exhaustivo comparando ambos instrumentos en características técnicas (settlement físico vs cash, tamaño del contrato, exercise style americano vs europeo, trading hours), aspectos operativos (liquidez y bid-ask spreads, costos de ejecución, margen requerido), consideraciones fiscales (tratamiento de ganancias/pérdidas, Sección 1256 para SPX vs Short Term Tax para SPY, impacto en impuestos), y casos de uso (¿cuándo usar SPY?, ¿cuándo usar SPX?, recomendaciones por tipo de trader).

Las diferencias clave incluyen: Settlement (SPY: físico/entrega de acciones, SPX: cash/liquidación en efectivo), Tamaño (SPY: 1x/100 shares, SPX: 10x/10x valor del índice), Exercise (SPY: americano/ejercicio anticipado, SPX: europeo/solo al vencimiento), Fiscalidad (SPY: Short Term Tax, SPX: 60/40 Sección 1256), Liquidez (SPY: muy alta, SPX: alta), y Spreads (SPY: muy estrechos, SPX: más amplios).

Las opciones sobre SPX califican para el tratamiento fiscal **60/40** bajo la Sección 1256 del código fiscal: 60% de ganancias/pérdidas se tratan como ganancias de capital a largo plazo (tasa ~15-20%), 40% se tratan como ganancias de capital a corto plazo (tasa ~37%), resultando en ahorro fiscal significativo vs SPY (todo se trata como corto plazo). Ejemplo práctico: Ganancia de $10,000 en SPX → Impuesto ~$2,200-2,600, misma ganancia en SPY → Impuesto ~$3,700, ahorro ~$1,100-1,500.

**Resultados esperados**: Tabla comparativa detallada de todas las características, análisis de casos de uso por tipo de trader, cálculo de impacto fiscal con ejemplos numéricos, recomendaciones específicas para diferentes escenarios, y guía de decisión: ¿SPY o SPX?



In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# OBJETIVO 2.6: COMPARACIÓN SPY vs SPX
# ═══════════════════════════════════════════════════════════════════════

comparison_data = {
    'Aspecto': [
        'Tipo de Instrumento',
        'Settlement',
        'Tamaño del Contrato',
        'Liquidez',
        'Bid-Ask Spread',
        'Tratamiento Fiscal',
        'Dividendos',
        'Exercise Style',
        'Trading Hours',
        'Costo de Ejecución',
        'Uso Principal',
        'Margen Requerido'
    ],
    'SPY (ETF)': [
        'ETF (Exchange Traded Fund)',
        'Physical (entrega de acciones)',
        '1x (100 shares por contrato)',
        'Muy Alta',
        'Muy Estrecho (~0.01%)',
        'STT (Short Term Tax)',
        'Incorporados en precio',
        'American (ejercicio anticipado)',
        'Regular (9:30-16:00 ET)',
        'Bajo',
        'Retail, Hedging',
        'Menor (margen de opciones)'
    ],
    'SPX (Índice)': [
        'Índice (S&P 500)',
        'Cash (liquidación en efectivo)',
        '10x (10x el valor del índice)',
        'Alta',
        'Más Amplio (~0.05%)',
        '60/40 (Sección 1256)',
        'No aplica (índice)',
        'European (solo al vencimiento)',
        'Incluye PM (hasta 16:15 ET)',
        'Moderado',
        'Institucional, Especulación',
        'Mayor (margen de índice)'
    ]
}

comparison_df = pd.DataFrame(comparison_data)

print(f"\n{'='*60}")
print(f"COMPARACIÓN DETALLADA: SPY vs SPX")
print(f"{'='*60}\n")

display(comparison_df.style.set_properties(**{'text-align': 'left'}))

# Análisis de casos de uso
print(f"\n{'='*60}")
print(f"ANÁLISIS DE CASOS DE USO")
print(f"{'='*60}\n")

use_cases = {
    'Retail Trading': {
        'SPY': '✓ Mejor opción: spreads estrechos, menor tamaño, ejercicio flexible',
        'SPX': '✗ Menos adecuado: tamaño grande, margen alto'
    },
    'Hedging de Portfolio': {
        'SPY': '✓ Adecuado: correlación alta, liquidez excelente',
        'SPX': '✓ También adecuado: mejor para portfolios grandes'
    },
    'Especulación Volatilidad': {
        'SPY': '✓ Bueno: liquidez alta permite entrada/salida rápida',
        'SPX': '✓ Excelente: tratamiento fiscal 60/40, cash settlement'
    },
    'Trading Institucional': {
        'SPY': '✓ Usado: para posiciones más pequeñas',
        'SPX': '✓ Preferido: tamaño grande, eficiencia fiscal'
    }
}

for use_case, analysis in use_cases.items():
    print(f"{use_case}:")
    print(f"  {analysis['SPY']}")
    print(f"  {analysis['SPX']}")
    print()

# Ventajas fiscales SPX
print(f"{'='*60}")
print(f"VENTAJAS FISCALES DE SPX (Sección 1256)")
print(f"{'='*60}")
print("""
Las opciones sobre SPX califican para tratamiento fiscal 60/40:
  - 60% de ganancias/pérdidas se tratan como ganancias de capital a largo plazo
  - 40% se tratan como ganancias de capital a corto plazo
  - Esto puede resultar en ahorro fiscal significativo vs SPY (todo STT)

Ejemplo:
  Ganancia de $10,000 en SPX:
    - $6,000 tratado como LTCG (tasa ~15-20%)
    - $4,000 tratado como STCG (tasa ~37%)
    - Impuesto total: ~$2,200-2,600
  
  Misma ganancia en SPY:
    - Todo tratado como STCG (tasa ~37%)
    - Impuesto total: ~$3,700
  
  Ahorro fiscal: ~$1,100-1,500
""")

# Recomendaciones
print(f"{'='*60}")
print(f"RECOMENDACIONES")
print(f"{'='*60}")
print("""
1. RETAIL TRADERS:
   → Usar SPY: mejor liquidez, spreads más estrechos, tamaño manejable

2. TRADERS INSTITUCIONALES:
   → Usar SPX: ventajas fiscales, tamaño eficiente, cash settlement

3. HEDGING:
   → SPY para portfolios pequeños/medianos
   → SPX para portfolios grandes o cuando se busca eficiencia fiscal

4. TRADING DE VOLATILIDAD:
   → Ambos son viables, pero SPX ofrece ventajas fiscales significativas
   → SPY puede ser mejor para scalping por spreads más estrechos
""")

print(f"{'='*60}\n")
print(" Objetivo 2.6 completado: Reflexión técnica SPY vs SPX")



## Objetivo 5: Neutralización de Delta con Otra Opción

### Descripción

Hasta ahora, hemos neutralizado el delta usando el subyacente (acciones de SPY). Sin embargo, también es posible neutralizar el delta de una opción usando **otra opción**. Este enfoque tiene implicaciones importantes en todas las griegas, no solo en el delta. Usar otra opción en lugar del subyacente ofrece flexibilidad (puedes elegir strikes y tipos de opción que modifiquen otras griegas), eficiencia de capital (a veces más eficiente que comprar/vender el subyacente), control de Gamma y Vega (puedes ajustar la exposición a estas griegas simultáneamente), y es la base para estrategias avanzadas como spreads, butterflies, condors.

Analizamos las consecuencias de neutralizar delta con otra opción: cálculo del ratio de cobertura (cuántas opciones de cobertura necesitamos), impacto en otras griegas (Gamma: ¿aumenta o disminuye?, Theta: ¿ganamos o perdemos tiempo?, Vega: ¿aumenta o disminuye la exposición a volatilidad?), y comparación con hedging con subyacente (ventajas y desventajas).

Para el cálculo del ratio de cobertura, si tenemos 1 opción principal con delta δ₁ y queremos usar otra opción con delta δ₂, necesitamos N opciones de cobertura tal que: δ₁ + N × δ₂ = 0, por lo tanto N = -δ₁ / δ₂. Analizamos griegas combinadas: delta combinado δ₁ + N × δ₂ (debe ser ≈ 0), gamma combinado γ₁ + N × γ₂, theta combinado θ₁ + N × θ₂, y vega combinado ν₁ + N × ν₂. Visualizamos el payoff diagram de la combinación y comparamos griegas antes y después.

**Funciones reutilizadas**: `bs_greeks_manual(S, K, T, r, q, sigma, right)` para calcular todas las griegas de ambas opciones.

**Ejemplo típico**: Opción principal Long Call ATM (delta ≈ 0.5), opción de cobertura Short Call OTM (delta ≈ 0.3), ratio -0.5 / 0.3 = -1.67 (necesitamos vender 1.67 Calls OTM).

**Implicaciones**: Gamma (si la opción de cobertura tiene gamma positivo, el gamma combinado puede aumentar), Theta (depende de si la opción de cobertura es long o short), Vega (similar a theta, depende de la posición), y Payoff (el payoff combinado es diferente al de una sola opción).

**Resultados esperados**: Cálculo del ratio de cobertura necesario, análisis de cómo cambian todas las griegas (Gamma, Vega, Theta), payoff diagram de la combinación, comparación visual de griegas antes/después, y recomendaciones sobre cuándo usar este método vs hedging con subyacente.



#  RESUMEN FINAL Y CONCLUSIONES

##  OBJETIVOS COMPLETADOS

### Clase 1 - Fundamentos y Análisis de Opciones:
1.  Conexión a un Broker desde Python (Interactive Brokers via ib_insync)
2.  Definir contratos de opciones sobre SPY y obtener cadenas de opciones completas
3.  Estimación de volatilidad implícita a partir de precios de mercado
4.  Visualización de volatility smiles y mini-superficie de volatilidad
5.  Calcular precio teórico y griegas (Δ, Γ, Θ, Vega, Rho) con implementación propia
6.  Representar evolución temporal histórica de opciones (griegas y payoff)
7.  Diseñar función de cobertura profesional y comparar posición cubierta vs expuesta
8.  Simular envío de órdenes y su cobertura, analizando problemas prácticos

### Clase 2 - Estrategias y Trading Avanzado:
1.  Construir estrategia Long Straddle periódico sobre SPY con backtesting
2.  Modificar para versión delta-hedged usando griegas propias calculadas
3.  Análisis exhaustivo de P&L histórico de ambas versiones
4.  Simular envío de órdenes: combo vs patas separadas, analizando riesgo de legging
5.  Neutralizar Delta con otra opción y analizar implicaciones
6.  Reflexión técnica: diferencias entre usar SPY vs SPX

##  APRENDIZAJES PRINCIPALES

1. **Implementación de Black-Scholes**: Las funciones standalone (`bs_price`, `bs_greeks_manual`) del código original son robustas y producen resultados altamente correlacionados con datos del broker.

2. **Delta Hedging**: El hedging reduce significativamente la volatilidad del P&L, pero los costos de transacción pueden erosionar beneficios en mercados de baja volatilidad.

3. **Volatilidad Implícita**: El "volatility smile" es más pronunciado en expiraciones cortas, y las opciones OTM (especialmente puts) muestran mayor volatilidad implícita.

4. **Estrategia Long Straddle**: Requiere movimientos significativos para ser rentable. El timing de entrada es crucial.

5. **SPY vs SPX**: La elección depende del tamaño de la posición, objetivos fiscales y tipo de trader (retail vs institucional).

##  NOTAS FINALES

- Todo el código nuevo **REUTILIZA** las funciones del código original del profesor
- La nomenclatura original (`S`, `K`, `T`, `r`, `q`, `sigma`) se mantiene en todo el código nuevo
- El código original está **PRESERVADO COMPLETAMENTE** sin modificaciones

---

**Proyecto completado exitosamente** 



## Objetivo 6: Reflexión Técnica - ¿Qué habría cambiado usando SPX en lugar de SPY?

### Descripción

Aunque SPY (ETF) y SPX (índice) representan el mismo subyacente económico (S&P 500), las opciones sobre cada uno tienen características muy diferentes que afectan significativamente la estrategia de trading, los costos, y los resultados fiscales. **SPY (SPDR S&P 500 ETF Trust)** es un ETF que replica el índice S&P 500, se negocia como una acción normal, y tiene opciones con ejercicio estilo americano. **SPX (S&P 500 Index)** es el índice mismo (no un instrumento negociable directamente), tiene opciones con ejercicio estilo europeo, y liquidación en efectivo (cash settlement).

Proporcionamos un análisis exhaustivo comparando ambos instrumentos en características técnicas (settlement físico vs cash, tamaño del contrato, exercise style americano vs europeo, trading hours), aspectos operativos (liquidez y bid-ask spreads, costos de ejecución, margen requerido), consideraciones fiscales (tratamiento de ganancias/pérdidas, Sección 1256 para SPX vs Short Term Tax para SPY, impacto en impuestos), y casos de uso (¿cuándo usar SPY?, ¿cuándo usar SPX?, recomendaciones por tipo de trader).

Las diferencias clave incluyen: Settlement (SPY: físico/entrega de acciones, SPX: cash/liquidación en efectivo), Tamaño (SPY: 1x/100 shares, SPX: 10x/10x valor del índice), Exercise (SPY: americano/ejercicio anticipado, SPX: europeo/solo al vencimiento), Fiscalidad (SPY: Short Term Tax, SPX: 60/40 Sección 1256), Liquidez (SPY: muy alta, SPX: alta), y Spreads (SPY: muy estrechos, SPX: más amplios).

Las opciones sobre SPX califican para el tratamiento fiscal **60/40** bajo la Sección 1256 del código fiscal: 60% de ganancias/pérdidas se tratan como ganancias de capital a largo plazo (tasa ~15-20%), 40% se tratan como ganancias de capital a corto plazo (tasa ~37%), resultando en ahorro fiscal significativo vs SPY (todo se trata como corto plazo). Ejemplo práctico: Ganancia de $10,000 en SPX → Impuesto ~$2,200-2,600, misma ganancia en SPY → Impuesto ~$3,700, ahorro ~$1,100-1,500.

**Resultados esperados**: Tabla comparativa detallada de todas las características, análisis de casos de uso por tipo de trader, cálculo de impacto fiscal con ejemplos numéricos, recomendaciones específicas para diferentes escenarios, y guía de decisión: ¿SPY o SPX?
